### <span style = "color:orange">1. Introduction<span>

#### <span style="color:green">1.1. Project purpose</span>

Xây dựng và đánh giá tập hợp các mô hình dự báo chuỗi thời gian để dự đoán nhiệt độ cực đại hàng ngày tại các khu vực đô thị và ven biển Việt Nam. Dự án áp dụng cả thuật toán Machine Learning truyền thống và Deep Learning hiện đại nhằm cải thiện độ chính xác dự báo so với các phương pháp thống kê thông thường.

<b>Nguồn dữ liệu</b>: Bộ dữ liệu ERA5 (ECMWF) với các bản ghi nhiệt độ từ 1990 đến 2024, được xử lý và biến đổi để huấn luyện các mô hình như Random Forest, XGBoost, LSTM, Transformer, TFT, và N-BEATS.

<b>Kết quả</b>: Đánh giá hiệu năng giữa các mô hình qua nhiều kịch bản thực nghiệm và đề xuất hệ thống cảnh báo nhiệt độ sớm ứng dụng thực tế.

#### <span style="color:green">1.2. Data source and description</span>

<h4>Thông tin dữ liệu trong đề tài:</h4>

<ul>
<li><b>Thời gian thu thập:</b> từ năm <b>1990 đến 2024</b></li>
<li><b>Định dạng ban đầu:</b> .grib, sau đó chuyển đổi sang .csv để xử lý</li>
</ul>

<h4>Các biến số chính trong tập dữ liệu:</h4>

<table>
<thead>
<tr>
<th>Tên cột dữ liệu</th>
<th>Ý nghĩa</th>
<th>Đơn vị đo</th>
</tr>
</thead>
<tbody>
<tr><td><code>NAME</code></td><td>Tên tỉnh/thành phố nơi thu thập dữ liệu</td><td>-</td></tr>
<tr><td><code>LATITUDE</code></td><td>Vĩ độ địa lý của điểm đo</td><td>Độ</td></tr>
<tr><td><code>LONGITUDE</code></td><td>Kinh độ địa lý của điểm đo</td><td>Độ</td></tr>
<tr><td><code>YMD</code></td><td>Ngày/tháng/năm đo đạc</td><td>dd/mm/yyyy</td></tr>
<tr><td><code>YEAR</code></td><td>Năm đo đạc</td><td>Năm</td></tr>
<tr><td><code>MONTH</code></td><td>Tháng đo đạc</td><td>Tháng</td></tr>
<tr><td><code>DAY</code></td><td>Ngày đo đạc</td><td>Ngày</td></tr>
<tr><td><code>TEMP_max</code></td><td>Nhiệt độ không khí cực đại trong ngày</td><td>°C</td></tr>
<tr><td><code>TEMP_ave</code></td><td>Nhiệt độ trung bình trong ngày</td><td>°C</td></tr>
<tr><td><code>DEW_ave</code></td><td>Điểm sương trung bình trong ngày</td><td>°C</td></tr>
<tr><td><code>DEW_max</code></td><td>Điểm sương cao nhất trong ngày</td><td>°C</td></tr>
<tr><td><code>RH_ave</code></td><td>Độ ẩm tương đối trung bình trong ngày</td><td>%</td></tr>
<tr><td><code>RH_max</code></td><td>Độ ẩm tương đối cực đại trong ngày</td><td>%</td></tr>
<tr><td><code>AT_ave</code></td><td>Nhiệt độ cảm nhận trung bình trong ngày (Apparent Temp.)</td><td>°C</td></tr>
<tr><td><code>AT_max</code></td><td>Nhiệt độ cảm nhận cao nhất trong ngày</td><td>°C</td></tr>
</tbody>
</table>

<p><b>Biến mục tiêu chính:</b></p>
<ul>
<li><code>TEMP_max</code> — Nhiệt độ không khí cực đại hàng ngày (°C)</li>
</ul>

<p><b>Lưu ý:</b> Dữ liệu gốc của ERA5 có thể chứa giá trị thiếu, giá trị ngoại lai và một số dị bản khí tượng đặc thù. Do đó, quá trình làm sạch dữ liệu, xử lý giá trị thiếu, phát hiện ngoại lệ và chuẩn hóa dữ liệu là các bước bắt buộc trước khi tiến hành huấn luyện và dự báo.</p>

In [ ]:
match_type = {
    'NAME'       : 'Categorical',        # Tên tỉnh/thành phố (chuỗi)
    'LATITUDE'   : 'Numerical',          # Vĩ độ (°)
    'LONGITUDE'  : 'Numerical',          # Kinh độ (°)
    'YMD'        : 'Datetime',           # Ngày/tháng/năm (dd/mm/yyyy)
    'YEAR'       : 'Numerical',          # Năm (năm)
    'MONTH'      : 'Numerical',          # Tháng (1-12)
    'DAY'        : 'Numerical',          # Ngày (1-31)

    'TEMP_max'   : 'Numerical',          # Nhiệt độ cực đại trong ngày (°C)
    'TEMP_ave'   : 'Numerical',          # Nhiệt độ trung bình trong ngày (°C)
    'DEW_ave'    : 'Numerical',          # Điểm sương trung bình trong ngày (°C)
    'DEW_max'    : 'Numerical',          # Điểm sương cực đại trong ngày (°C)
    'RH_ave'     : 'Numerical',          # Độ ẩm tương đối trung bình trong ngày (%)
    'RH_max'     : 'Numerical',          # Độ ẩm tương đối cực đại trong ngày (%)
    'AT_ave'     : 'Numerical',          # Nhiệt độ cảm nhận trung bình trong ngày (°C)
    'AT_max'     : 'Numerical'           # Nhiệt độ cảm nhận cực đại trong ngày (°C)
}

#### <span style="color:green">1.3. Goals</span>

<img src="../image/Ảnh chụp màn hình 2025-06-24 210616.png">

### <span style="color:orange">2. Import Libraries</span>

#### <span style="color:green">2.1. Configuration and display settings</span>

In [ ]:
import sys
sys.path.append("../")  # đường dẫn đến thư mục chứa src

from src.utilities import(config, 
                          dataset, 
                          features, 
                          plots)

from src.models import(anomaly_models,
                       forecasting_models,
                       model_utils)

#### <span style="color:green">2.2. Required Python packages</span>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import metpy
import metpy.calc
import tensorflow as tf

from sklearn.model_selection import train_test_split
from metpy.units import units
from copy        import deepcopy

from sklearn.ensemble  import IsolationForest
from prophet           import Prophet
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster   import AgglomerativeClustering
from sklearn.cluster   import DBSCAN
from hdbscan           import HDBSCAN
from joblib            import Memory

### <span style="color:orange">3.Data Collecting</span>

#### <span style="color:green">3.1. Chuyển các file .grib sang file .csv hoặc .xlsx	, hoặc .xls (file Excel)</span>

In [ ]:
src_CaMau = dict({"CaMau_SurPres"     : list(["../data/raw/Ca Mau/.grib/CM_SurPres_1990_2002.grib",
                                              "../data/raw/Ca Mau/.grib/CM_SurPres_2003_2015.grib",
                                              "../data/raw/Ca Mau/.grib/CM_SurPres_2016_2024.grib"]),
                  "CaMau_TotalCloud"  : list(["../data/raw/Ca Mau/.grib/CM_TotalCloud_1990_2002.grib",
                                              "../data/raw/Ca Mau/.grib/CM_TotalCloud_2003_2015.grib",
                                              "../data/raw/Ca Mau/.grib/CM_TotalCloud_2016_2024.grib"]),
                  "CaMau_TotalPre"    : list(["../data/raw/Ca Mau/.grib/CM_TotalPre_1990_2002.grib",
                                              "../data/raw/Ca Mau/.grib/CM_TotalPre_2003_2015.grib",
                                              "../data/raw/Ca Mau/.grib/CM_TotalPre_2016_2024.grib"]),
                  "CaMau_u10"         : list(["../data/raw/Ca Mau/.grib/CM_u10_1990_2002.grib",
                                              "../data/raw/Ca Mau/.grib/CM_u10_2003_2015.grib",
                                              "../data/raw/Ca Mau/.grib/CM_u10_2016_2024.grib"]),
                  "CaMau_v10"         : list(["../data/raw/Ca Mau/.grib/CM_v10_1990_2002.grib",
                                              "../data/raw/Ca Mau/.grib/CM_v10_2003_2015.grib",
                                              "../data/raw/Ca Mau/.grib/CM_v10_2016_2024.grib"])})


src_DongHoi = dict({"DongHoi_SurPres"     : list(["../data/raw/Dong Hoi/.grib/DH_SurfacePre_1990_2002.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_SurfacePre_2003_2015.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_SurfacePre_2016_2024.grib"]),
                    "DongHoi_TotalCloud"  : list(["../data/raw/Dong Hoi/.grib/DH_TotalCloud_1990_2002.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_TotalCloud_2003_2015.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_TotalCloud_2016_2024.grib"]),
                    "DongHoi_TotalPre"    : list(["../data/raw/Dong Hoi/.grib/DH_TotalPre_1990_2002.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_TotalPre_2003_2015.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_TotalPre_2016_2024.grib"]),
                    "DongHoi_u10"         : list(["../data/raw/Dong Hoi/.grib/DH_u10_1990_2002.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_u10_2003_2015.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_u10_2016_2024.grib"]),
                    "DongHoi_v10"         : list(["../data/raw/Dong Hoi/.grib/DH_v10_1990_2002.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_v10_2003_2015.grib",
                                                  "../data/raw/Dong Hoi/.grib/DH_v10_2016_2024.grib"])})


src_NoiBai = dict({"NB_SurPres"       : list(["../data/raw/Noi Bai/.grib/NB_SurPres_1990_2002.grib",
                                              "../data/raw/Noi Bai/.grib/NB_SurPres_2003_2015.grib",
                                              "../data/raw/Noi Bai/.grib/NB_SurPres_2016_2024.grib"]),
                   "NB_TotalCloud"    : list(["../data/raw/Noi Bai/.grib/NB_TotalCloud_1990_2002.grib",
                                              "../data/raw/Noi Bai/.grib/NB_TotalCloud_2003_2015.grib",
                                              "../data/raw/Noi Bai/.grib/NB_TotalCloud_2016_2024.grib"]),
                   "NB_TotalPre"      : list(["../data/raw/Noi Bai/.grib/NB_TotalPre_1990_2002.grib",
                                              "../data/raw/Noi Bai/.grib/NB_TotalPre_2003_2015.grib",
                                              "../data/raw/Noi Bai/.grib/NB_TotalPre_2016_2024.grib"]),
                   "NB_u10"           : list(["../data/raw/Noi Bai/.grib/NB_u10_1990_2002.grib",
                                              "../data/raw/Noi Bai/.grib/NB_u10_2003_2015.grib",
                                              "../data/raw/Noi Bai/.grib/NB_u10_2016_2024.grib"]),
                   "NB_v10"           : list(["../data/raw/Noi Bai/.grib/NB_v10_1990_2002.grib",
                                              "../data/raw/Noi Bai/.grib/NB_v10_2003_2015.grib",
                                              "../data/raw/Noi Bai/.grib/NB_v10_2016_2024.grib"])})


src_QuyNhon = dict({"QN_SurPres"      : list(["../data/raw/Quy Nhon/.grib/QN_SurPres_1990_2002.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_SurPres_2003_2015.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_SurPres_2016_2024.grib"]),
                    "QN_TotalCloud"   : list(["../data/raw/Quy Nhon/.grib/QN_TotalCloud_1990_2002.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_TotalCloud_2003_2015.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_TotalCloud_2016_2024.grib"]),
                    "QN_TotalPre"     : list(["../data/raw/Quy Nhon/.grib/QN_TotalPre_1990_2002.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_TotalPre_2003_2015.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_TotalPre_2016_2024.grib"]),
                    "QN_u10"          : list(["../data/raw/Quy Nhon/.grib/QN_u10_1990_2002.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_u10_2003_2015.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_u10_2016_2024.grib"]),
                    "QN_v10"          : list(["../data/raw/Quy Nhon/.grib/QN_v10_1990_2002.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_v10_2003_2015.grib",
                                              "../data/raw/Quy Nhon/.grib/QN_v10_2016_2024.grib"])})


src_ThanhHoa = dict({"TH_SurPres"         : list(["../data/raw/Thanh Hoa/.grib/TH_SurPres_1990_2002.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_SurPres_2003_2015.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_SurPres_2016_2024.grib"]),
                     "TH_TotalCloud"      : list(["../data/raw/Thanh Hoa/.grib/TH_TotalCloud_1990_2002.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_TotalCloud_2003_2015.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_TotalCloud_2016_2024.grib"]),
                     "TH_TotalPre"        : list(["../data/raw/Thanh Hoa/.grib/TH_TotalPre_1990_2002.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_TotalPre_2003_2015.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_TotalPre_2016_2024.grib"]),
                     "TH_u10"             : list(["../data/raw/Thanh Hoa/.grib/TH_u10_1990_2002.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_u10_2003_2015.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_u10_2016_2024.grib"]),
                     "TH_v10"             : list(["../data/raw/Thanh Hoa/.grib/TH_v10_1990_2002.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_v10_2003_2015.grib",
                                                  "../data/raw/Thanh Hoa/.grib/TH_v10_2016_2024.grib"])})


src_TSN = dict({"TSN_SurPres"         : list(["../data/raw/TSN/.grib/TSN_Press_1990_2002.grib",
                                              "../data/raw/TSN/.grib/TSN_Press_2003_2015.grib",
                                              "../data/raw/TSN/.grib/TSN_Press_2016_2024.grib"]),
                "TSN_TotalCloud"      : list(["../data/raw/TSN/.grib/TSN_TotalCloud_1990_2002.grib",
                                              "../data/raw/TSN/.grib/TSN_TotalCloud_2003_2015.grib",
                                              "../data/raw/TSN/.grib/TSN_TotalCloud_2016_2024.grib"]),
                "TSN_TotalPre"        : list(["../data/raw/TSN/.grib/TSN_TotalPre_1990_2002.grib",
                                              "../data/raw/TSN/.grib/TSN_TotalPre_2003_2015.grib",
                                              "../data/raw/TSN/.grib/TSN_TotalPre_2016_2024.grib"]),
                "TSN_u10"             : list(["../data/raw/TSN/.grib/TSN_u10_1990_2002.grib",
                                              "../data/raw/TSN/.grib/TSN_u10_2003_2015.grib",
                                              "../data/raw/TSN/.grib/TSN_u10_2016-2023.grib"]),
                "TSN_v10"             : list(["../data/raw/TSN/.grib/TSN_v10_1990_2002.grib",
                                              "../data/raw/TSN/.grib/TSN_v10_2003_2015.grib",
                                              "../data/raw/TSN/.grib/TSN_v10_2016_2023.grib"])})

src = src_CaMau | src_DongHoi | src_NoiBai | src_QuyNhon  | src_ThanhHoa | src_TSN

In [ ]:
for var_name, file_list in src.items():
    for file_path in file_list:
        # Lấy thư mục chứa file .grib
        grib_folder = os.path.dirname(file_path)

        # Thư mục .csv cùng cấp với .grib
        csv_folder = os.path.join(os.path.dirname(grib_folder), '.csv')
        os.makedirs(csv_folder, exist_ok=True)

        # Tên file .csv trùng với tên file .grib nhưng đổi đuôi
        file_name = os.path.basename(file_path).replace(".grib", ".csv")

        # File đích nằm trong thư mục .csv
        des_file = os.path.join(csv_folder, file_name)

        # Nếu chưa tồn tại mới convert
        if not os.path.exists(des_file):
            print(f"~> Convert: {file_path} → {des_file}")
            dataset.grib_to_csv(file_path, des_file)
        else:
            print(f"✔️ Tồn tại: {des_file}, bỏ qua.")

#### <span style="color:green">3.2. Tiếp theo, đổi thời gian trong cột "time" (UTC+00) của file vừa mới chuyển về giờ Việt Nam (UTC+07)</span>

In [ ]:
for var_name, file_list in src.items():
    print(f"\n🔸 Đang xử lý biến: {var_name}")

    # Lấy thư mục chứa file csv gốc (từ file đầu tiên thôi)
    csv_path_example = file_list[0].replace(".grib", ".csv")
    csv_folder = os.path.dirname(csv_path_example)
    merged_folder = os.path.join(csv_folder, "merged")
    os.makedirs(merged_folder, exist_ok=True)

    # Tên file đích
    out_path = os.path.join(merged_folder, f"{var_name}_merged.csv")

    # Nếu file đích đã tồn tại thì bỏ qua
    if os.path.exists(out_path):
        print(f"⚠️ Đã tồn tại: {out_path}, bỏ qua.")
        continue

    df_list = []  # List lưu các DataFrame để concat

    for file_path in file_list:
        csv_path = file_path.replace(".grib", ".csv")

        if not os.path.exists(csv_path):
            print(f"❌ Không tìm thấy file: {csv_path}, bỏ qua.")
            continue

        df = pd.read_csv(csv_path)
        df = features.utc_to_vietnam(df, column_name="time")
        df_list.append(df)
        print(f"✔️ Đã xử lý: {csv_path}")

    # Nếu không có file nào thì bỏ qua merge
    if not df_list:
        print(f"⚠️ Không có dữ liệu cho {var_name}, bỏ qua.")
        continue

    # Nối tất cả DataFrame lại với nhau
    df_merged = pd.concat(df_list, ignore_index=True)

    # Lưu kết quả
    df_merged.to_csv(out_path, index=False)
    print(f"✅ Đã lưu: {out_path}")

#### <span style="color:green">3.3. Ghép các file khí tượng vào file nhiệt độ để thành data đầy đủ cho model</span>

Các cách loại bỏ dòng trùng trong pandas

<table border="1" cellpadding="6" cellspacing="0">
<thead>
<tr>
<th>Cách dùng</th>
<th>Mô tả</th>
<th>Ví dụ</th>
</tr>
</thead>
<tbody>

<tr>
<td><code>df.drop_duplicates()</code></td>
<td>Loại bỏ toàn bộ các dòng bị trùng tất cả giá trị ở tất cả các cột.</td>
<td><pre>df.drop_duplicates()</pre></td>
</tr>

<tr>
<td><code>df.drop_duplicates(subset=["col1"])</code></td>
<td>Chỉ kiểm tra trùng lặp theo cột <code>col1</code>, giữ lại dòng đầu tiên.</td>
<td><pre>df.drop_duplicates(subset=["time"])</pre></td>
</tr>

<tr>
<td><code>df.drop_duplicates(keep="last")</code></td>
<td>Giữ lại dòng cuối cùng trong số các dòng bị trùng.</td>
<td><pre>df.drop_duplicates(subset=["time"], keep="last")</pre></td>
</tr>

<tr>
<td><code>df.drop_duplicates(ignore_index=True)</code></td>
<td>Xóa dòng trùng, đồng thời reset lại chỉ số (index).</td>
<td><pre>df.drop_duplicates(ignore_index=True)</pre></td>
</tr>

<tr>
<td><code>df.loc[~df.index.duplicated() ]</code></td>
<td>Nếu <code>time</code> đang là index, loại bỏ các index trùng, giữ lại dòng đầu tiên.</td>
<td><pre>df.loc[~df.index.duplicated()]</pre></td>
</tr>

<tr>
<td><code>df.loc[~df.index.duplicated(keep="last") ]</code></td>
<td>Loại bỏ các index trùng, giữ lại dòng cuối cùng.</td>
<td><pre>df.loc[~df.index.duplicated(keep="last")]</pre></td>
</tr>

</tbody>
</table>


In [ ]:
primary_key = ["time", "latitude", "longitude", "number", "step", "surface"]

##### <span style="color:tomato">Ca Mau</span>

SurPres

In [ ]:
df_CaMau_SurPres = pd.read_csv("../data/raw/Ca Mau/.csv/merged/CaMau_SurPres_merged.csv")
df_CaMau_SurPres = df_CaMau_SurPres.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols             = primary_key + ["sp"]
df_CaMau_SurPres = df_CaMau_SurPres[cols]

df_CaMau_SurPres

TotalCloud

In [ ]:
df_CaMau_TotalCloud = pd.read_csv("../data/raw/Ca Mau/.csv/merged/CaMau_TotalCloud_merged.csv")
df_CaMau_TotalCloud = df_CaMau_TotalCloud.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols                = primary_key + ["tcc"]
df_CaMau_TotalCloud = df_CaMau_TotalCloud[cols]

df_CaMau_TotalCloud

TotalPre

In [ ]:
df_CaMau_TotalPre = pd.read_csv("../data/raw/Ca Mau/.csv/merged/CaMau_TotalPre_merged.csv")
df_CaMau_TotalPre = df_CaMau_TotalPre.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols              = primary_key + ["tp"]
df_CaMau_TotalPre = df_CaMau_TotalPre[cols]


df_CaMau_TotalPre

In [ ]:
"""
    Chuyển time = time + step
    Chuyển step = 0 days
"""
df_CaMau_TotalPre["time"] = pd.to_datetime(df_CaMau_TotalPre["time"]) + pd.to_timedelta(df_CaMau_TotalPre["step"])
df_CaMau_TotalPre["step"] = "0 days"

df_CaMau_TotalPre

u10

In [ ]:
df_CaMau_u10 = pd.read_csv("../data/raw/Ca Mau/.csv/merged/CaMau_u10_merged.csv")
df_CaMau_u10 = df_CaMau_u10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols         = primary_key + ["u10"]
df_CaMau_u10 = df_CaMau_u10[cols]

df_CaMau_u10

v10

In [ ]:
df_CaMau_v10 = pd.read_csv("../data/raw/Ca Mau/.csv/merged/CaMau_v10_merged.csv")
df_CaMau_v10 = df_CaMau_v10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols         = primary_key + ["v10"]
df_CaMau_v10 = df_CaMau_v10[cols]

df_CaMau_v10

merge file

In [ ]:
datasets = list([df_CaMau_SurPres, df_CaMau_TotalCloud, df_CaMau_TotalPre, df_CaMau_u10, df_CaMau_v10])

for df in datasets:
    df = features.utc_to_vietnam(df,"time")

df_merged = dataset.merge_df(datasets = datasets,
                             on       = primary_key, 
                             how      = "outer")
df_merged

In [ ]:
dataset.compare_df(df_A = df_merged,
                   df_B = df_CaMau_TotalPre,
                   key  = primary_key)

In [ ]:
df_merged.to_csv("../data/raw/Ca Mau/.csv/merged/CaMau_merged.csv", index=False)

##### <span style="color:tomato">Dong Hoi</span>

SurPres

In [ ]:
df_DH_SurPres = pd.read_csv("../data/raw//Dong Hoi/.csv/merged/DongHoi_SurPres_merged.csv")
df_DH_SurPres = df_DH_SurPres.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols             = primary_key + ["sp"]
df_DH_SurPres = df_DH_SurPres[cols]

df_DH_SurPres

TotalCloud

In [ ]:
df_DH_TotalCloud = pd.read_csv("../data/raw/Dong Hoi/.csv/merged/DongHoi_TotalCloud_merged.csv")
df_DH_TotalCloud = df_DH_TotalCloud.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols                = primary_key + ["tcc"]
df_DH_TotalCloud = df_DH_TotalCloud[cols]

df_DH_TotalCloud

TotalPre

In [ ]:
df_DH_TotalPre = pd.read_csv("../data/raw/Dong Hoi/.csv/merged/DongHoi_TotalPre_merged.csv")
df_DH_TotalPre = df_DH_TotalPre.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols           = primary_key + ["tp"]
df_DH_TotalPre = df_DH_TotalPre[cols]

df_DH_TotalPre

In [ ]:
"""
    Chuyển time = time + step
    Chuyển step = 0 days
"""
df_DH_TotalPre["time"] = pd.to_datetime(df_DH_TotalPre["time"]) + pd.to_timedelta(df_DH_TotalPre["step"])
df_DH_TotalPre["step"] = "0 days"

df_DH_TotalPre

u10

In [ ]:
df_DH_u10 = pd.read_csv("../data/raw/Dong Hoi/.csv/merged/DongHoi_u10_merged.csv")
df_DH_u10 = df_DH_u10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols         = primary_key + ["u10"]
df_DH_u10 = df_DH_u10[cols]

df_DH_u10

v10

In [ ]:
df_DH_v10 = pd.read_csv("../data/raw/Dong Hoi/.csv/merged/DongHoi_v10_merged.csv")
df_DH_v10 = df_DH_v10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols         = primary_key + ["v10"]
df_DH_v10 = df_DH_v10[cols]

df_DH_v10

merge file

In [ ]:
datasets = list([df_DH_SurPres, df_DH_TotalCloud, df_DH_TotalPre, df_DH_u10, df_DH_v10])

for df in datasets:
    df = features.utc_to_vietnam(df,"time")

df_merged = dataset.merge_df(datasets = datasets,
                             on       = primary_key, 
                             how      = "outer")
df_merged

In [ ]:
dataset.compare_df(df_A = df_merged,
                   df_B = df_DH_TotalPre,
                   key  = primary_key)

In [ ]:
df_merged.to_csv("../data/raw/Dong Hoi/.csv/merged/DH_merged.csv", index=False)

##### <span style="color:tomato">Noi Bai</span>

SurPres

In [ ]:
df_NB_SurPres = pd.read_csv("../data/raw/Noi Bai/.csv/merged/NB_SurPres_merged.csv")
df_NB_SurPres = df_NB_SurPres.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols          = primary_key + ["sp"]
df_NB_SurPres = df_NB_SurPres[cols]

df_NB_SurPres

TotalCloud

In [ ]:
df_NB_TotalCloud = pd.read_csv("../data/raw/Noi Bai/.csv/merged/NB_TotalCloud_merged.csv")
df_NB_TotalCloud = df_NB_TotalCloud.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols             = primary_key + ["tcc"]
df_NB_TotalCloud = df_NB_TotalCloud[cols]

df_NB_TotalCloud

TotalPre

In [ ]:
df_NB_TotalPre = pd.read_csv("../data/raw/Noi Bai/.csv/merged/NB_TotalPre_merged.csv")
# df_CaMau_TotalPre = df_CaMau_TotalPre.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols           = primary_key + ["tp"]
df_NB_TotalPre = df_NB_TotalPre[cols]

df_NB_TotalPre

In [ ]:
"""
    Chuyển time = time + step
    Chuyển step = 0 days
"""
df_NB_TotalPre["time"] = pd.to_datetime(df_NB_TotalPre["time"]) + pd.to_timedelta(df_NB_TotalPre["step"])
df_NB_TotalPre["step"] = "0 days"

df_NB_TotalPre

u10

In [ ]:
df_NB_u10 = pd.read_csv("../data/raw/Noi Bai/.csv/merged/NB_u10_merged.csv")
df_NB_u10 = df_NB_u10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols      = primary_key + ["u10"]
df_NB_u10 = df_NB_u10[cols]

df_NB_u10

v10

In [ ]:
df_NB_v10 = pd.read_csv("../data/raw/Noi Bai/.csv/merged/NB_v10_merged.csv")
df_NB_v10 = df_NB_v10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols      = primary_key + ["v10"]
df_NB_v10 = df_NB_v10[cols]

df_NB_v10

merge file

In [ ]:
datasets = list([df_NB_SurPres, df_NB_TotalCloud, df_NB_TotalPre, df_NB_u10, df_NB_v10])

for df in datasets:
    df = features.utc_to_vietnam(df,"time")

df_merged = dataset.merge_df(datasets = datasets,
                             on       = primary_key, 
                             how      = "outer")
df_merged

In [ ]:
dataset.compare_df(df_A = df_merged,
                   df_B = df_NB_TotalPre,
                   key  = primary_key)

In [ ]:
df_merged.to_csv("../data/raw/Noi Bai/.csv/merged/NB_merged.csv", index=False)

##### <span style="color:tomato">Quy Nhon</span>

SurPres

In [ ]:
df_QN_SurPres = pd.read_csv("../data/raw/Quy Nhon/.csv/merged/QN_SurPres_merged.csv")
df_QN_SurPres = df_QN_SurPres.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols          = primary_key + ["sp"]
df_QN_SurPres = df_QN_SurPres[cols]

df_QN_SurPres

TotalCloud

In [ ]:
df_QN_TotalCloud = pd.read_csv("../data/raw/Quy Nhon/.csv/merged/QN_TotalCloud_merged.csv")
df_QN_TotalCloud = df_QN_TotalCloud.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols             = primary_key + ["tcc"]
df_QN_TotalCloud = df_QN_TotalCloud[cols]

df_QN_TotalCloud

TotalPre

In [ ]:
df_QN_TotalPre = pd.read_csv("../data/raw/Quy Nhon/.csv/merged/QN_TotalPre_merged.csv")
df_QN_TotalPre = df_QN_TotalPre.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols           = primary_key + ["tp"]
df_QN_TotalPre = df_QN_TotalPre[cols]

df_QN_TotalPre

In [ ]:
"""
    Chuyển time = time + step
    Chuyển step = 0 days
"""
df_QN_TotalPre["time"] = pd.to_datetime(df_QN_TotalPre["time"]) + pd.to_timedelta(df_QN_TotalPre["step"])
df_QN_TotalPre["step"] = "0 days"

df_QN_TotalPre

u10

In [ ]:
df_QN_u10 = pd.read_csv("../data/raw/Quy Nhon/.csv/merged/QN_u10_merged.csv")
df_QN_u10 = df_QN_u10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols      = primary_key + ["u10"]
df_QN_u10 = df_QN_u10[cols]

df_QN_u10

v10

In [ ]:
df_QN_v10 = pd.read_csv("../data/raw/Quy Nhon/.csv/merged/QN_v10_merged.csv")
df_QN_v10 = df_QN_v10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols      = primary_key + ["v10"]
df_QN_v10 = df_QN_v10[cols]

df_QN_v10

merge file

In [ ]:
datasets = list([df_QN_SurPres, df_QN_TotalCloud, df_QN_TotalPre, df_QN_u10, df_QN_v10])

for df in datasets:
    df = features.utc_to_vietnam(df,"time")

df_merged = dataset.merge_df(datasets = datasets,
                             on       = primary_key, 
                             how      = "outer")
df_merged

In [ ]:
dataset.compare_df(df_A = df_merged,
                   df_B = df_QN_TotalPre,
                   key  = primary_key)

In [ ]:
df_merged.to_csv("../data/raw/Quy Nhon/.csv/merged/QN_merged.csv", index=False)

##### <span style="color:tomato">Thanh Hoa</span>

SurPres

In [ ]:
df_TH_SurPres = pd.read_csv("../data/raw/Thanh Hoa/.csv/merged/TH_SurPres_merged.csv")
df_TH_SurPres = df_TH_SurPres.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols             = primary_key + ["sp"]
df_TH_SurPres = df_TH_SurPres[cols]

df_TH_SurPres

TotalCloud

In [ ]:
df_TH_TotalCloud = pd.read_csv("../data/raw/Thanh Hoa/.csv/merged/TH_TotalCloud_merged.csv")
df_TH_TotalCloud = df_TH_TotalCloud.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols                = primary_key + ["tcc"]
df_TH_TotalCloud = df_TH_TotalCloud[cols]

df_TH_TotalCloud

TotalPre

In [ ]:
df_TH_TotalPre = pd.read_csv("../data/raw/Thanh Hoa/.csv/merged/TH_TotalPre_merged.csv")
df_TH_TotalPre = df_TH_TotalPre.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols              = primary_key + ["tp"]
df_TH_TotalPre = df_TH_TotalPre[cols]

df_TH_TotalPre

In [ ]:
"""
    Chuyển time = time + step
    Chuyển step = 0 days
"""
df_TH_TotalPre["time"] = pd.to_datetime(df_TH_TotalPre["time"]) + pd.to_timedelta(df_TH_TotalPre["step"])
df_TH_TotalPre["step"] = "0 days"

df_TH_TotalPre

u10

In [ ]:
df_TH_u10 = pd.read_csv("../data/raw/Thanh Hoa/.csv/merged/TH_u10_merged.csv")
df_TH_u10 = df_TH_u10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols         = primary_key + ["u10"]
df_TH_u10 = df_TH_u10[cols]

df_TH_u10

v10

In [ ]:
df_TH_v10 = pd.read_csv("../data/raw/Thanh Hoa/.csv/merged/TH_v10_merged.csv")
df_TH_v10 = df_TH_v10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols         = primary_key + ["v10"]
df_TH_v10 = df_TH_v10[cols]

df_TH_v10

merge file

In [ ]:
datasets = list([df_TH_SurPres, df_TH_TotalCloud, df_TH_TotalPre, df_TH_u10, df_TH_v10])

for df in datasets:
    df = features.utc_to_vietnam(df,"time")

df_merged = dataset.merge_df(datasets = datasets,
                             on       = primary_key, 
                             how      = "outer")
df_merged

In [ ]:
dataset.compare_df(df_A = df_merged,
                   df_B = df_TH_TotalPre,
                   key  = primary_key)

In [ ]:
df_merged.to_csv("../data/raw/Thanh Hoa/.csv/merged/TH_merged.csv", index=False)

##### <span style="color:tomato">TSN</span>

SurPres

In [ ]:
df_TSN_SurPres = pd.read_csv("../data/raw/TSN/.csv/merged/TSN_SurPres_merged.csv")
df_TSN_SurPres = df_TSN_SurPres.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols           = primary_key + ["sp"]
df_TSN_SurPres = df_TSN_SurPres[cols]

df_TSN_SurPres

TotalCloud

In [ ]:
df_TSN_TotalCloud = pd.read_csv("../data/raw/TSN/.csv/merged/TSN_TotalCloud_merged.csv")
df_TSN_TotalCloud = df_TSN_TotalCloud.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols              = primary_key + ["tcc"]
df_TSN_TotalCloud = df_TSN_TotalCloud[cols]

df_TSN_TotalCloud

TotalPre

In [ ]:
df_TSN_TotalPre = pd.read_csv("../data/raw/TSN/.csv/merged/TSN_TotalPre_merged.csv")
df_TSN_TotalPre = df_TSN_TotalPre.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols            = primary_key + ["tp"]
df_TSN_TotalPre = df_TSN_TotalPre[cols]

df_TSN_TotalPre

In [ ]:
"""
    Chuyển time = time + step
    Chuyển step = 0 days
"""
df_TSN_TotalPre["time"] = pd.to_datetime(df_TSN_TotalPre["time"]) + pd.to_timedelta(df_TSN_TotalPre["step"])
df_TSN_TotalPre["step"] = "0 days"

df_TSN_TotalPre

u10

In [ ]:
df_TSN_u10 = pd.read_csv("../data/raw/TSN/.csv/merged/TSN_u10_merged.csv")
df_TSN_u10 = df_TSN_u10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols       = primary_key + ["u10"]
df_TSN_u10 = df_TSN_u10[cols]

df_TSN_u10

v10

In [ ]:
df_TSN_v10 = pd.read_csv("../data/raw/TSN/.csv/merged/TSN_v10_merged.csv")
df_TSN_v10 = df_TSN_v10.drop("valid_time", axis=1).drop_duplicates()

# Sắp xếp lại thứ tự cột
cols       = primary_key + ["v10"]
df_TSN_v10 = df_TSN_v10[cols]

df_TSN_v10

merge file

In [ ]:
datasets = list([df_TSN_SurPres, df_TSN_TotalCloud, df_TSN_TotalPre, df_TSN_u10, df_TSN_v10])

for df in datasets:
    df = features.utc_to_vietnam(df,"time")

df_merged = dataset.merge_df(datasets = datasets,
                             on       = primary_key, 
                             how      = "outer")
df_merged

In [ ]:
dataset.compare_df(df_A = df_merged,
                   df_B = df_TSN_TotalPre,
                   key  = primary_key)

In [ ]:
df_merged.to_csv("../data/raw/TSN/.csv/merged/TSN_merged.csv", index=False)

### <span style="color:orange">4.Data Cleaning</span>

Các biến số chính trong tập dữ liệu:

<table>
<thead>
<tr>
<th>Tên cột dữ liệu</th>
<th>Ý nghĩa</th>
<th>Đơn vị đo</th>
</tr>
</thead>
<tbody>
<tr><td><code>sp</code></td><td>Áp suất khí quyển tại mặt đất</td><td>Pa</td></tr>
<tr><td><code>tcc</code></td><td>Tổng độ che phủ mây toàn phần</td><td>%</td></tr>
<tr><td><code>tp</code></td><td>Lượng mưa tích lũy</td><td>mm</td></tr>
<tr><td><code>u10</code></td><td>Thành phần vận tốc gió theo trục Đông-Tây ở độ cao 10m</td><td>m/s</td></tr>
<tr><td><code>v10</code></td><td>Thành phần vận tốc gió theo trục Bắc-Nam ở độ cao 10m</td><td>m/s</td></tr>
</tbody>
</table>


In [ ]:
numerical_features = ["sp", "tcc", "tp", "u10", "v10"]

#### <span style="color:green">4.1. Handling duplicate values</span>

In [ ]:
src = list(["../data/raw/Ca Mau/.csv/merged/CaMau_merged.csv",
            "../data/raw/Dong Hoi/.csv/merged/DH_merged.csv",
            "../data/raw/Noi Bai/.csv/merged/NB_merged.csv",
            "../data/raw/Quy Nhon/.csv/merged/QN_merged.csv",
            "../data/raw/Thanh Hoa/.csv/merged/TH_merged.csv",
            "../data/raw/TSN/.csv/merged/TSN_merged.csv"])

station = dict({})

In [ ]:
for idx, val in enumerate(src):
    df = pd.read_csv(val, parse_dates=["time"])
    station_name = os.path.basename(val).replace("_merged.csv", "")
    station[station_name] = df  # lưu vào dict theo tên trạm
    print(f"🔸 Trạm: {station_name}")

    dataset.ProportionDuplicate_aproach1(df["time"])
    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")

    station[name] = dataset.HandleDuplicate_drop(data   = df,
                                                 subset = "time",
                                                 keep   = "first")
    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")
    
    dataset.ProportionDuplicate_aproach1(df["time"])
    print()

#### <span style="color:green">4.2. Handling missing values</span>

##### <span style="color:tomato">Index</span>

<a href="https://datastud.dev/posts/time-series-resample">Filling Gaps in Time Series Data</a>

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")

    gap_df = dataset.find_time_gaps(data       = df.set_index("time"), 
                                    start_time = "1990-01-01 00:00:00+07:00", 
                                    end_time   = "2025-01-01 00:00:00+07:00",
                                    freq       = "1h")
    print(gap_df)
    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")
    
    station[name] = dataset.fill_time_gaps(data       = df.set_index("time"), 
                                           start_time = "1990-01-01 00:00:00+07:00", 
                                           end_time   = "2025-01-01 00:00:00+07:00",
                                           freq       = "1h")
    print(f"✅ Đã xong")

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")

    gap_df = dataset.find_time_gaps(data       = df.set_index("time"), 
                                    start_time = "1990-01-01 00:00:00+07:00", 
                                    end_time   = "2025-01-01 00:00:00+07:00",
                                    freq       = "1h")
    print(gap_df)
    print()

##### <span style="color:tomato">Features</span>

📊 So sánh các phương pháp xử lý giá trị thiếu

<table>
<thead>
<tr>
<th>Phương pháp</th>
<th>Ưu điểm</th>
<th>Nhược điểm</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>dropna()</code></td>
<td>Nhanh gọn, triệt để</td>
<td>Mất dữ liệu</td>
</tr>
<tr>
<td><code>fillna(mean/median)</code></td>
<td>Dễ thực hiện, hợp với dữ liệu số</td>
<td>Có thể làm giảm phương sai</td>
</tr>
<tr>
<td><code>ffill()</code> / <code>bfill()</code></td>
<td>Dùng tốt với time series</td>
<td>Có thể tạo bias</td>
</tr>
<tr>
<td><code>interpolate()</code></td>
<td>Tự nhiên, giữ xu hướng</td>
<td>Cần dữ liệu liên tục</td>
</tr>
<tr>
<td><code>groupby fill</code></td>
<td>Phân nhóm thông minh</td>
<td>Chậm với dữ liệu lớn</td>
</tr>
</tbody>
</table>

✅ Kết luận:
<ul>
<li><b>Time series</b> → <code>interpolate()</code>, <code>ffill()</code></li>
<li><b>Dữ liệu nhóm</b> → <code>groupby fill</code></li>
<li><b>Dữ liệu có ít NaN</b> → <code>dropna()</code></li>
<li><b>Dữ liệu số nhiều NaN</b> → <code>fillna(mean/median)</code> hoặc <code>interpolate()</code></li>
</ul>


| Thứ tự | Phương pháp                                              | Áp dụng khi                                                                   | Ưu tiên                                                                 |
| :----- | :------------------------------------------------------- | :---------------------------------------------------------------------------- | :---------------------------------------------------------------------- |
| **1**  | `interpolate(method="time")`                             | Dữ liệu liên tục, time series, các biến số khí tượng biến động theo thời gian | ✅ Ưu tiên đầu                                                           |
| **2**  | `ffill()` / `bfill()`                                    | Nếu `interpolate` không fill hết, dùng giá trị trước/sau gần nhất             | ⚠️ Chỉ nên dùng khi dữ liệu không có sự biến động lớn hoặc missing ngắn |
| **3**  | `groupby fill` (theo ngày, tháng, năm hoặc theo station) | Nếu dữ liệu phân nhóm và mỗi nhóm đủ dày                                      | Tốt cho dữ liệu dạng panel, hoặc khí tượng từng trạm                    |
| **4**  | `fillna(mean/median)`                                    | Nếu số lượng missing còn nhỏ lẻ, hoặc sau khi các cách trên vẫn còn missing   | Dùng được với numerical không quá nhạy cảm                              |
| **5**  | `dropna()`                                               | Khi số lượng missing ít, hoặc không thể nội suy mà giữ lại sẽ gây sai lệch    | Cuối cùng                                                               |


In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")

    dataset.ProportionMissing_aproach1(df[numerical_features])
    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")
    
    # Set DatetimeIndex
    station[name] = df.set_index("time")

    # Nội suy các biến numerical
    df_filled = dataset.HandleMissing_interpolate(data   = station[name], 
                                                  method = "time")

    # Gán lại vào DataFrame gốc 
    station[name] = df_filled.reset_index()

    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")

    dataset.ProportionMissing_aproach1(df[numerical_features])
    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")
    
    # Set DatetimeIndex
    station[name] = df.set_index("time")

    # Nội suy các biến numerical
    df_filled = dataset.HandleMissing_fillna(data   = station[name], 
                                             method = "median")

    # Gán lại vào DataFrame gốc 
    station[name] = df_filled.reset_index()

    print()

In [ ]:
for name, df in station.items():
    print(f"🔸 Trạm: {name}")

    dataset.ProportionMissing_aproach1(df[numerical_features])
    print()

In [ ]:
station["CaMau"]

#### <span style="color:green">4.3. Handling mismatch values</span>

#### <span style="color:green">4.4. Handling abnormal <~> outlier values <u>(basic)</u></span>

📖 Tổng hợp cách xử lý outlier cho Time Series
| Phương pháp                                  | Mô tả                                                                             | Khi nào dùng                                  | Ưu điểm        | Nhược điểm                           |
| :------------------------------------------- | :-------------------------------------------------------------------------------- | :-------------------------------------------- | :------------- | :----------------------------------- |
| **Remove**                                   | Loại các điểm dữ liệu ngoài ngưỡng                                                | Nếu rõ ràng là lỗi đo đạc                     | Dễ làm         | Mất dữ liệu, đứt chuỗi               |
| **Interpolation**                            | Nội suy giá trị tại vị trí outlier bằng trung bình giữa trước và sau              | Khi outlier là lỗi cục bộ                     | Giữ mượt chuỗi | Không đúng nếu outlier là event thực |
| **LOCF (Last Observation Carried Forward)**  | Lấy giá trị trước đó thay thế                                                     | Dữ liệu nhạy biến động thấp (ví dụ: nhiệt độ) | Dễ cài         | Nếu outlier đúng là event, bị sai    |
| **NOCB (Next Observation Carried Backward)** | Lấy giá trị tiếp theo thay thế                                                    | Tương tự LOCF                                 | Dễ làm         | Sai với biến thiên nhanh             |
| **Smoothing (Rolling Mean / EWMA)**          | Làm mượt chuỗi, giảm ảnh hưởng outlier bằng trung bình trượt                      | Nếu cần dự báo dài hạn                        | Giảm nhiễu     | Mất tín hiệu ngắn hạn                |
| **Model-based Detection**                    | Dùng Isolation Forest, Prophet changepoint detection… phát hiện outlier rồi xử lý | Khi data nhiều outlier hoặc nhiều mùa vụ      | Nhận diện tốt  | Tốn thời gian                        |
| **Capping theo percentile/IQR từng đoạn**    | Chỉ capping giá trị tại vị trí bất thường                                         | Nếu không muốn interpolate                    | Giữ data       | Mất thông tin cực trị                |


📊 Hướng xử lý theo loại outlier trong time series
| Loại outlier                            | Cách xử lý tốt                        |
| :-------------------------------------- | :------------------------------------ |
| **Lỗi nhập liệu, spike dị thường**      | Remove / LOCF / Interpolate           |
| **Số liệu sự kiện thực (mưa lớn, bão)** | Giữ nguyên                            |
| **Nhiễu nhẹ / bất thường nhỏ**          | Rolling Mean / EWMA smoothing         |
| **Đứt đoạn giá trị**                    | Interpolation (linear/spline)         |
| **Spike vào dịp lễ, sự kiện (như tết)** | Có thể tạo biến indicator thay vì xóa |



- <a href="https://towardsdatascience.com/the-ultimate-guide-to-finding-outliers-in-your-time-series-data-part-1-1bf81e09ade4-2/">The Ultimate Guide to Finding Outliers in Your Time-Series Data (Part 1)</a> <--> Phương pháp xác định (visual/statistical)
- <a href="https://medium.com/data-science/the-ultimate-guide-to-finding-outliers-in-your-time-series-data-part-2-674c25837f29">The Ultimate Guide to Finding Outliers in Your Time-Series Data (Part 2)</a> <--> Sử dụng machine learning
- <a href="https://towardsdatascience.com/the-ultimate-guide-to-finding-outliers-in-your-time-series-data-part-3-0ff73ce28ca3-2/">The Ultimate Guide to Finding Outliers in Your Time-Series Data (Part 3)</a> <--> “Outliers Found: Now What?” – quản lý outliers
- <a href="https://towardsdatascience.com/evaluating-the-impact-of-outlier-treatment-in-time-series-b4fac4cabe94/">The Ultimate Guide to Finding Outliers in Your Time-Series Data (Part 4)</a>

📖 Các giá trị freq thường dùng trong resample() Pandas:
| Tần suất       | `freq` string        | Ý nghĩa                                   |
| :------------- | :------------------- | :---------------------------------------- |
| **Hàng giây**  | `"S"`                | Mỗi giây                                  |
| **Hàng phút**  | `"T"` (hoặc `"min"`) | Mỗi phút                                  |
| **Hàng giờ**   | `"H"`                | Mỗi giờ                                   |
| **Hàng ngày**  | `"D"`                | Mỗi ngày                                  |
| **Hàng tuần**  | `"W"`                | Mỗi tuần (mặc định kết thúc vào Chủ nhật) |
| **Hàng tháng** | `"M"`                | Cuối mỗi tháng                            |
| **Đầu tháng**  | `"MS"`               | Đầu mỗi tháng                             |
| **Hàng quý**   | `"Q"`                | Cuối mỗi quý (3, 6, 9, 12)                |
| **Đầu quý**    | `"QS"`               | Đầu mỗi quý                               |
| **Hàng năm**   | `"A"` hoặc `"Y"`     | Cuối mỗi năm (mặc định 31-12)             |
| **Đầu năm**    | `"AS"` hoặc `"YS"`   | Đầu mỗi năm (1-1)                         |


In [ ]:
station["CaMau"].loc[:, ["time", "v10"]].head(20)

In [ ]:
new_df = station["CaMau"].loc[:, ["time", "v10"]].set_index("time").resample("1H").mean().interpolate()
new_df.head(20)

##### <span style="color:tomato">Ca Mau</span>

In [ ]:
plots.plot_feature_trends_over_time(data         = station["CaMau"],
                                    data_cols    = numerical_features,
                                    station_name = "CaMau",
                                    start_time   = "1990-01-01 00:00:00+07:00",
                                    end_time     = "2025-01-01 00:00:00+07:00",
                                    freq         = "1Y")

detect

In [ ]:
plots.plot_feature_outliers_over_time(data         = station["CaMau"],
                                      data_cols    = numerical_features,
                                      station_name = "CaMau",
                                      method       = "statistic",
                                      display      = False,
                                      start_time   = "1990-01-01 00:00:00+07:00",
                                      end_time     = "2025-01-01 00:00:00+07:00",
                                      freq         = "1H")

handle

In [ ]:
station["CaMau"] = dataset.handle_feature_outliers_over_time(data         = station["CaMau"],
                                                             data_cols    = numerical_features,
                                                             station_name = "CaMau",
                                                             method       = "statistic",
                                                             display      = True,
                                                             start_time   = "1990-01-01 00:00:00+07:00",
                                                             end_time     = "2025-01-01 00:00:00+07:00",
                                                             freq         = "1H")

evaluate

##### <span style="color:tomato">Dong Hoi</span>

In [ ]:
plots.plot_feature_trends_over_time(data         = station["DH"],
                                    data_cols    = numerical_features,
                                    station_name = "DH",
                                    start_time   = "1990-01-01 00:00:00+07:00",
                                    end_time     = "2025-01-01 00:00:00+07:00",
                                    freq         = "1Y")

detect

In [ ]:
plots.plot_feature_outliers_over_time(data         = station["DH"],
                                      data_cols    = numerical_features,
                                      station_name = "DH",
                                      method       = "statistic",
                                      display      = False,
                                      start_time   = "1990-01-01 00:00:00+07:00",
                                      end_time     = "2025-01-01 00:00:00+07:00",
                                      freq         = "1H")

handle

In [ ]:
station["DH"] = dataset.handle_feature_outliers_over_time(data         = station["DH"],
                                                          data_cols    = numerical_features,
                                                          station_name = "DH",
                                                          method       = "statistic",
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">Noi Bai</span>

In [ ]:
plots.plot_feature_trends_over_time(data         = station["NB"],
                                    data_cols    = numerical_features,
                                    station_name = "NB",
                                    start_time   = "1990-01-01 00:00:00+07:00",
                                    end_time     = "2025-01-01 00:00:00+07:00",
                                    freq         = "1Y")

detect

In [ ]:
plots.plot_feature_outliers_over_time(data         = station["NB"],
                                      data_cols    = numerical_features,
                                      station_name = "NB",
                                      method       = "statistic",
                                      display      = False,
                                      start_time   = "1990-01-01 00:00:00+07:00",
                                      end_time     = "2025-01-01 00:00:00+07:00",
                                      freq         = "1H")

handle

In [ ]:
station["NB"] = dataset.handle_feature_outliers_over_time(data         = station["NB"],
                                                          data_cols    = numerical_features,
                                                          station_name = "NB",
                                                          method       = "statistic",
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">Quy Nhon</span>

In [ ]:
plots.plot_feature_trends_over_time(data         = station["QN"],
                                    data_cols    = numerical_features,
                                    station_name = "QN",
                                    start_time   = "1990-01-01 00:00:00+07:00",
                                    end_time     = "2025-01-01 00:00:00+07:00",
                                    freq         = "1Y")

detect

In [ ]:
plots.plot_feature_outliers_over_time(data         = station["QN"],
                                      data_cols    = numerical_features,
                                      station_name = "QN",
                                      method       = "statistic",
                                      display      = False,
                                      start_time   = "1990-01-01 00:00:00+07:00",
                                      end_time     = "2025-01-01 00:00:00+07:00",
                                      freq         = "1H")

handle

In [ ]:
station["QN"] = dataset.handle_feature_outliers_over_time(data         = station["QN"],
                                                          data_cols    = numerical_features,
                                                          station_name = "QN",
                                                          method       = "statistic",
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">Thanh Hoa</span>

In [ ]:
plots.plot_feature_trends_over_time(data         = station["TH"],
                                    data_cols    = numerical_features,
                                    station_name = "TH",
                                    start_time   = "1990-01-01 00:00:00+07:00",
                                    end_time     = "2025-01-01 00:00:00+07:00",
                                    freq         = "1Y")

detect

In [ ]:
plots.plot_feature_outliers_over_time(data         = station["TH"],
                                      data_cols    = numerical_features,
                                      station_name = "TH",
                                      method       = "statistic",
                                      display      = False,
                                      start_time   = "1990-01-01 00:00:00+07:00",
                                      end_time     = "2025-01-01 00:00:00+07:00",
                                      freq         = "1H")

handle

In [ ]:
station["TH"] = dataset.handle_feature_outliers_over_time(data         = station["TH"],
                                                          data_cols    = numerical_features,
                                                          station_name = "TH",
                                                          method       = "statistic",
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">TSN</span>

In [ ]:
plots.plot_feature_trends_over_time(data         = station["TSN"],
                                    data_cols    = numerical_features,
                                    station_name = "TSN",
                                    start_time   = "1990-01-01 00:00:00+07:00",
                                    end_time     = "2025-01-01 00:00:00+07:00",
                                    freq         = "1Y")

detect

In [ ]:
plots.plot_feature_outliers_over_time(data         = station["TSN"],
                                      data_cols    = numerical_features,
                                      station_name = "TSN",
                                      method       = "statistic",
                                      display      = False,
                                      start_time   = "1990-01-01 00:00:00+07:00",
                                      end_time     = "2025-01-01 00:00:00+07:00",
                                      freq         = "1H")

handle

In [ ]:
station["TSN"] = dataset.handle_feature_outliers_over_time(data         = station["TSN"],
                                                           data_cols    = numerical_features,
                                                           station_name = "TSN",
                                                           method       = "statistic",
                                                           display      = True,
                                                           start_time   = "1990-01-01 00:00:00+07:00",
                                                           end_time     = "2025-01-01 00:00:00+07:00",
                                                           freq         = "1H")

evaluate

#### <span style="color:green">4.5. Save data</span>

In [ ]:
station["TSN"]

In [ ]:
# Tạo thư mục nếu chưa có
os.makedirs("../data/processed/datasets", exist_ok=True)

# Duyệt từng station và lưu file
for name, df in station.items():
    file_name = f"../data/processed/datasets/{name}90.24_cleaned.csv"
    df.to_csv(file_name, index=False)
    print(f"Đã lưu: {file_name}")

### <span style="color:orange">5. Data Preprocessing</span>

#### <span style="color:green">5.1. Handling abnormal <~> outlier values <u>(intermediate)</u></span>

📌 1️⃣ So sánh Univariate Time Series và Multivariate Time Series
| Tiêu chí               | **Univariate Time Series**                      | **Multivariate Time Series**                                                 |
| :--------------------- | :---------------------------------------------- | :--------------------------------------------------------------------------- |
| **Định nghĩa**         | Dự báo 1 biến duy nhất theo thời gian           | Dự báo 1 hoặc nhiều biến, có thể phụ thuộc thêm các biến khác theo thời gian |
| **Đầu vào (features)** | Chỉ có giá trị của chính biến đó theo thời gian | Có thể bao gồm nhiều biến liên quan (covariates) cùng thời gian              |
| **Độ phức tạp**        | Thấp hơn, dễ triển khai                         | Cao hơn do tính đến mối quan hệ giữa các biến                                |
| **Ví dụ**              | Dự báo nhiệt độ từng ngày tại 1 thành phố       | Dự báo nhiệt độ từng ngày + độ ẩm, tốc độ gió, kinh độ/vĩ độ, ngày lễ…       |
| **Kỹ thuật áp dụng**   | ARIMA, Prophet, LSTM, N-BEATS, Moving Average…  | VAR, LSTM với nhiều input, TFT, Multivariate N-BEATS…                        |
| **Ưu điểm**            | Dữ liệu và mô hình đơn giản, nhanh chóng        | Có thể tận dụng mối quan hệ giữa các biến để dự báo chính xác hơn            |
| **Nhược điểm**         | Không tận dụng được các yếu tố ảnh hưởng khác   | Cần nhiều dữ liệu hơn, xử lý phức tạp, dễ gặp vấn đề với outlier             |


📌 2️⃣ Cách xử lý outlier trong multivariate time series
| Phương pháp                                     | Cách thực hiện                                                                    | Ưu điểm                                 | Nhược điểm                                     |
| :---------------------------------------------- | :-------------------------------------------------------------------------------- | :-------------------------------------- | :--------------------------------------------- |
| **Z-score / IQR cho từng biến**                 | Tính z-score hoặc IQR cho từng feature, loại bỏ giá trị vượt ngưỡng               | Dễ triển khai, đơn giản                 | Chỉ xử lý được từng biến, không xét tương quan |
| **Multivariate Z-score (Mahalanobis distance)** | Tính khoảng cách Mahalanobis giữa mỗi điểm và phân phối tổng thể                  | Xét được mối quan hệ giữa các biến      | Cần giả định phân phối dữ liệu gần chuẩn       |
| **Isolation Forest / One-Class SVM**            | Dùng mô hình unsupervised để phát hiện điểm bất thường trên toàn bộ feature space | Tự động, không cần giả định phân phối   | Chậm với dữ liệu lớn, tuning khó               |
| **Autoencoder / LSTM-AE**                       | Train autoencoder, điểm nào reconstruction error lớn thì là outlier               | Phù hợp time series, capture non-linear | Cần nhiều dữ liệu, training lâu                |
| **Visual inspection (scatter matrix, PCA)**     | Giảm số chiều xuống 2D/3D để trực quan và phát hiện outlier                       | Nhanh, trực quan                        | Khó áp dụng khi dữ liệu quá lớn                |


📌 3️⃣ Gợi ý cho bài toán của bạn:

Với bài toán dự báo nhiệt độ cực đại theo thời gian có geo-feature

→ Nên dùng Isolation Forest hoặc Autoencoder để phát hiện outlier multivariate vì:

- Có thể capture mối quan hệ giữa nhiệt độ, độ ẩm, kinh độ, vĩ độ, thành phố
- Không cần giả định phân phối chuẩn
- Hợp cho dữ liệu nhiều biến dạng time series

Nếu dataset vừa phải thì dùng Isolation Forest
Nếu dataset lớn hoặc quan hệ non-linear mạnh thì dùng LSTM-AE.

📌 4️⃣ Tóm lại

- Univariate chỉ với 1 biến — đơn giản nhưng không tận dụng được yếu tố khác
- Multivariate phức tạp hơn, nhiều thông tin hơn nhưng phải xử lý outlier đúng cách.

Outlier Multivariate nên xử lý bằng Isolation Forest, Mahalanobis Distance hoặc Autoencoder để xét tương quan nhiều chiều.

---

📖 Các giá trị freq thường dùng trong resample() Pandas:
| Tần suất       | `freq` string        | Ý nghĩa                                   |
| :------------- | :------------------- | :---------------------------------------- |
| **Hàng giây**  | `"S"`                | Mỗi giây                                  |
| **Hàng phút**  | `"T"` (hoặc `"min"`) | Mỗi phút                                  |
| **Hàng giờ**   | `"H"`                | Mỗi giờ                                   |
| **Hàng ngày**  | `"D"`                | Mỗi ngày                                  |
| **Hàng tuần**  | `"W"`                | Mỗi tuần (mặc định kết thúc vào Chủ nhật) |
| **Hàng tháng** | `"M"`                | Cuối mỗi tháng                            |
| **Đầu tháng**  | `"MS"`               | Đầu mỗi tháng                             |
| **Hàng quý**   | `"Q"`                | Cuối mỗi quý (3, 6, 9, 12)                |
| **Đầu quý**    | `"QS"`               | Đầu mỗi quý                               |
| **Hàng năm**   | `"A"` hoặc `"Y"`     | Cuối mỗi năm (mặc định 31-12)             |
| **Đầu năm**    | `"AS"` hoặc `"YS"`   | Đầu mỗi năm (1-1)                         |


In [ ]:
# plots.plot_feature_trends_over_time(data         = station["CaMau"],
#                                     data_cols    = numerical_features,
#                                     station_name = "CaMau",
#                                     start_time   = "1990-01-01 00:00:00+07:00",
#                                     end_time     = "2025-01-01 00:00:00+07:00",
#                                     freq         = "1M")

##### <span style="color:tomato">CaMau</span>

detect

In [ ]:
# plots.plot_feature_outliers_over_time(data         = station["CaMau"],
#                                       data_cols    = numerical_features,
#                                       station_name = "CaMau",
#                                       method       = "machine_learning",
#                                       models       = dict({
#                                                            "IsolationForest"         : IsolationForest(bootstrap     = False, 
#                                                                                                        contamination = "auto", 
#                                                                                                        max_features  = 1.0, 
#                                                                                                        max_samples   = "auto", 
#                                                                                                        n_estimators  = 100, 
#                                                                                                        n_jobs        = None, 
#                                                                                                        random_state  = None, 
#                                                                                                        verbose       = 0, 
#                                                                                                        warm_start    = False),
#                                                            "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
#                                                                                                           contamination = "auto", 
#                                                                                                           leaf_size     = 30, 
#                                                                                                           metric        = "minkowski", 
#                                                                                                           metric_params = None, 
#                                                                                                           n_jobs        = None, 
#                                                                                                           n_neighbors   = 20, 
#                                                                                                           novelty       = False, 
#                                                                                                           p             = 2),
#                                                            "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
#                                                                                                changepoints            = None,
#                                                                                                n_changepoints          = 25,
#                                                                                                changepoint_range       = 0.8,
#                                                                                                yearly_seasonality      = "auto",
#                                                                                                weekly_seasonality      = "auto",
#                                                                                                daily_seasonality       = "auto",
#                                                                                                holidays                = None,
#                                                                                                seasonality_mode        = "additive",
#                                                                                                seasonality_prior_scale = 10.0,
#                                                                                                holidays_prior_scale    = 10.0,
#                                                                                                changepoint_prior_scale = 0.05,
#                                                                                                mcmc_samples            = 0,
#                                                                                                interval_width          = 0.80,
#                                                                                                uncertainty_samples     = 1000,
#                                                                                                stan_backend            = None),
#                                                            "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
#                                                                                                                compute_full_tree  = 'auto', 
#                                                                                                             #    connectivity       = None, # KHONG SUA
#                                                                                                                distance_threshold = None, 
#                                                                                                             #    linkage            = 'ward', # KHONG SUA
#                                                                                                                memory             = None, 
#                                                                                                                metric             = 'euclidean', 
#                                                                                                                n_clusters         = 3),
#                                                            "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
#                                                                                                                        min_samples                    = None,
#                                                                                                                        metric                         = 'euclidean',
#                                                                                                                        alpha                          = 1.0,
#                                                                                                                        p                              = None,
#                                                                                                                        algorithm                      = 'best',
#                                                                                                                        leaf_size                      = 40,
#                                                                                                                        memory                         = Memory(location=None),
#                                                                                                                        approx_min_span_tree           = True,
#                                                                                                                        gen_min_span_tree              = False,
#                                                                                                                        core_dist_n_jobs               = 1,
#                                                                                                                        cluster_selection_method       = 'eom',
#                                                                                                                        allow_single_cluster           = False,
#                                                                                                                        prediction_data                = False,
#                                                                                                                        match_reference_implementation = False),
#                                                            "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
#                                                            }),
#                                       factor       = 1.5,   # Stick with Prophet model
#                                       window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
#                                       dendrogram   = False, # Stick with Agglomerative Clustering model
#                                       display      = False,
#                                       start_time   = "1990-01-01 00:00:00+07:00",
#                                       end_time     = "2025-01-01 00:00:00+07:00",
#                                       freq         = "1H")

handle

In [ ]:
from joblib import Memory


station["CaMau"] = dataset.handle_feature_outliers_over_time(data         = station["CaMau"],
                                                             data_cols    = numerical_features,
                                                             station_name = "CaMau",
                                                             method       = "machine_learning",
                                                             models       = dict({
                                                                                  "IsolationForest"         : IsolationForest(bootstrap     = False, 
                                                                                                                              contamination = "auto", 
                                                                                                                              max_features  = 1.0, 
                                                                                                                              max_samples   = "auto", 
                                                                                                                              n_estimators  = 100, 
                                                                                                                              n_jobs        = None, 
                                                                                                                              random_state  = None, 
                                                                                                                              verbose       = 0, 
                                                                                                                              warm_start    = False),
                                                                                  "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
                                                                                                                                 contamination = "auto", 
                                                                                                                                 leaf_size     = 30, 
                                                                                                                                 metric        = "minkowski", 
                                                                                                                                 metric_params = None, 
                                                                                                                                 n_jobs        = None, 
                                                                                                                                 n_neighbors   = 20, 
                                                                                                                                 novelty       = False, 
                                                                                                                                 p             = 2),
                                                                                  "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
                                                                                                                      changepoints            = None,
                                                                                                                      n_changepoints          = 25,
                                                                                                                      changepoint_range       = 0.8,
                                                                                                                      yearly_seasonality      = "auto",
                                                                                                                      weekly_seasonality      = "auto",
                                                                                                                      daily_seasonality       = "auto",
                                                                                                                      holidays                = None,
                                                                                                                      seasonality_mode        = "additive",
                                                                                                                      seasonality_prior_scale = 10.0,
                                                                                                                      holidays_prior_scale    = 10.0,
                                                                                                                      changepoint_prior_scale = 0.05,
                                                                                                                      mcmc_samples            = 0,
                                                                                                                      interval_width          = 0.80,
                                                                                                                      uncertainty_samples     = 1000,
                                                                                                                      stan_backend            = None),
                                                                                  "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
                                                                                                                                      compute_full_tree  = 'auto', 
                                                                                                                                   #    connectivity       = None, # KHONG SUA
                                                                                                                                      distance_threshold = None, 
                                                                                                                                   #    linkage            = 'ward', # KHONG SUA
                                                                                                                                      memory             = None, 
                                                                                                                                      metric             = 'euclidean', 
                                                                                                                                      n_clusters         = 3),
                                                                                  "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
                                                                                                                       min_samples                    = None,
                                                                                                                       metric                         = 'euclidean',
                                                                                                                       alpha                          = 1.0,
                                                                                                                       p                              = None,
                                                                                                                       algorithm                      = 'best',
                                                                                                                       leaf_size                      = 40,
                                                                                                                       memory                         = Memory(location=None),
                                                                                                                       approx_min_span_tree           = True,
                                                                                                                       gen_min_span_tree              = False,
                                                                                                                       core_dist_n_jobs               = 1,
                                                                                                                       cluster_selection_method       = 'eom',
                                                                                                                       allow_single_cluster           = False,
                                                                                                                       prediction_data                = False,
                                                                                                                       match_reference_implementation = False),
                                                                                  "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
                                                                                  }),
                                                             factor       = 1.5,   # Stick with Prophet model
                                                             window_size  = 10,    # Stick with Agglomerative Clustering & HDBSCAN model
                                                             dendrogram   = False, # Stick with Agglomerative Clustering model
                                                             display      = True,
                                                             start_time   = "1990-01-01 00:00:00+07:00",
                                                             end_time     = "2025-01-01 00:00:00+07:00",
                                                             freq         = "1H")

evaluate

##### <span style="color:tomato">Dong Hoi</span>

detect

In [ ]:
# plots.plot_feature_outliers_over_time(data         = station["DH"],
#                                       data_cols    = numerical_features,
#                                       station_name = "DongHoi",
#                                       method       = "machine_learning",
#                                       models       = dict({
#                                                            "IsolationForest"         : IsolationForest(bootstrap     = False, 
#                                                                                                        contamination = "auto", 
#                                                                                                        max_features  = 1.0, 
#                                                                                                        max_samples   = "auto", 
#                                                                                                        n_estimators  = 100, 
#                                                                                                        n_jobs        = None, 
#                                                                                                        random_state  = None, 
#                                                                                                        verbose       = 0, 
#                                                                                                        warm_start    = False),
#                                                            "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
#                                                                                                           contamination = "auto", 
#                                                                                                           leaf_size     = 30, 
#                                                                                                           metric        = "minkowski", 
#                                                                                                           metric_params = None, 
#                                                                                                           n_jobs        = None, 
#                                                                                                           n_neighbors   = 20, 
#                                                                                                           novelty       = False, 
#                                                                                                           p             = 2),
#                                                            "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
#                                                                                                changepoints            = None,
#                                                                                                n_changepoints          = 25,
#                                                                                                changepoint_range       = 0.8,
#                                                                                                yearly_seasonality      = "auto",
#                                                                                                weekly_seasonality      = "auto",
#                                                                                                daily_seasonality       = "auto",
#                                                                                                holidays                = None,
#                                                                                                seasonality_mode        = "additive",
#                                                                                                seasonality_prior_scale = 10.0,
#                                                                                                holidays_prior_scale    = 10.0,
#                                                                                                changepoint_prior_scale = 0.05,
#                                                                                                mcmc_samples            = 0,
#                                                                                                interval_width          = 0.80,
#                                                                                                uncertainty_samples     = 1000,
#                                                                                                stan_backend            = None),
#                                                            "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
#                                                                                                                compute_full_tree  = 'auto', 
#                                                                                                             #    connectivity       = None, # KHONG SUA
#                                                                                                                distance_threshold = None, 
#                                                                                                             #    linkage            = 'ward', # KHONG SUA
#                                                                                                                memory             = None, 
#                                                                                                                metric             = 'euclidean', 
#                                                                                                                n_clusters         = 3),
#                                                            "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
#                                                                                                                        min_samples                    = None,
#                                                                                                                        metric                         = 'euclidean',
#                                                                                                                        alpha                          = 1.0,
#                                                                                                                        p                              = None,
#                                                                                                                        algorithm                      = 'best',
#                                                                                                                        leaf_size                      = 40,
#                                                                                                                        memory                         = Memory(location=None),
#                                                                                                                        approx_min_span_tree           = True,
#                                                                                                                        gen_min_span_tree              = False,
#                                                                                                                        core_dist_n_jobs               = 1,
#                                                                                                                        cluster_selection_method       = 'eom',
#                                                                                                                        allow_single_cluster           = False,
#                                                                                                                        prediction_data                = False,
#                                                                                                                        match_reference_implementation = False),
#                                                            "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
#                                                            }),
#                                       factor       = 1.5,   # Stick with Prophet model
#                                       window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
#                                       dendrogram   = False, # Stick with Agglomerative Clustering model
#                                       display      = False,
#                                       start_time   = "1990-01-01 00:00:00+07:00",
#                                       end_time     = "2025-01-01 00:00:00+07:00",
#                                       freq         = "1H")

handle

In [ ]:
station["DH"] = dataset.handle_feature_outliers_over_time(data         = station["DH"],
                                                          data_cols    = numerical_features,
                                                          station_name = "DongHoi",
                                                          method       = "machine_learning",
                                                          models       = dict({
                                                                               "IsolationForest"         : IsolationForest(bootstrap     = False, 
                                                                                                                           contamination = "auto", 
                                                                                                                           max_features  = 1.0, 
                                                                                                                           max_samples   = "auto", 
                                                                                                                           n_estimators  = 100, 
                                                                                                                           n_jobs        = None, 
                                                                                                                           random_state  = None, 
                                                                                                                           verbose       = 0, 
                                                                                                                           warm_start    = False),
                                                                               "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
                                                                                                                              contamination = "auto", 
                                                                                                                              leaf_size     = 30, 
                                                                                                                              metric        = "minkowski", 
                                                                                                                              metric_params = None, 
                                                                                                                              n_jobs        = None, 
                                                                                                                              n_neighbors   = 20, 
                                                                                                                              novelty       = False, 
                                                                                                                              p             = 2),
                                                                               "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
                                                                                                                   changepoints            = None,
                                                                                                                   n_changepoints          = 25,
                                                                                                                   changepoint_range       = 0.8,
                                                                                                                   yearly_seasonality      = "auto",
                                                                                                                   weekly_seasonality      = "auto",
                                                                                                                   daily_seasonality       = "auto",
                                                                                                                   holidays                = None,
                                                                                                                   seasonality_mode        = "additive",
                                                                                                                   seasonality_prior_scale = 10.0,
                                                                                                                   holidays_prior_scale    = 10.0,
                                                                                                                   changepoint_prior_scale = 0.05,
                                                                                                                   mcmc_samples            = 0,
                                                                                                                   interval_width          = 0.80,
                                                                                                                   uncertainty_samples     = 1000,
                                                                                                                   stan_backend            = None),
                                                                               "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
                                                                                                                                   compute_full_tree  = 'auto', 
                                                                                                                                #    connectivity       = None, # KHONG SUA
                                                                                                                                   distance_threshold = None, 
                                                                                                                                #    linkage            = 'ward', # KHONG SUA
                                                                                                                                   memory             = None, 
                                                                                                                                   metric             = 'euclidean', 
                                                                                                                                   n_clusters         = 3),
                                                                               "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
                                                                                                                       min_samples                    = None,
                                                                                                                       metric                         = 'euclidean',
                                                                                                                       alpha                          = 1.0,
                                                                                                                       p                              = None,
                                                                                                                       algorithm                      = 'best',
                                                                                                                       leaf_size                      = 40,
                                                                                                                       memory                         = Memory(location=None),
                                                                                                                       approx_min_span_tree           = True,
                                                                                                                       gen_min_span_tree              = False,
                                                                                                                       core_dist_n_jobs               = 1,
                                                                                                                       cluster_selection_method       = 'eom',
                                                                                                                       allow_single_cluster           = False,
                                                                                                                       prediction_data                = False,
                                                                                                                       match_reference_implementation = False),
                                                                               "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
                                                                               }),
                                                          factor       = 1.5,   # Stick with Prophet model
                                                          window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
                                                          dendrogram   = False, # Stick with Agglomerative Clustering model
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">Noi Bai</span>

detect

In [ ]:
# plots.plot_feature_outliers_over_time(data         = station["NB"],
#                                       data_cols    = numerical_features,
#                                       station_name = "NoiBai",
#                                       method       = "machine_learning",
#                                       models       = dict({
#                                                            "IsolationForest"         : IsolationForest(bootstrap     = False, 
#                                                                                                        contamination = "auto", 
#                                                                                                        max_features  = 1.0, 
#                                                                                                        max_samples   = "auto", 
#                                                                                                        n_estimators  = 100, 
#                                                                                                        n_jobs        = None, 
#                                                                                                        random_state  = None, 
#                                                                                                        verbose       = 0, 
#                                                                                                        warm_start    = False),
#                                                            "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
#                                                                                                           contamination = "auto", 
#                                                                                                           leaf_size     = 30, 
#                                                                                                           metric        = "minkowski", 
#                                                                                                           metric_params = None, 
#                                                                                                           n_jobs        = None, 
#                                                                                                           n_neighbors   = 20, 
#                                                                                                           novelty       = False, 
#                                                                                                           p             = 2),
#                                                            "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
#                                                                                                changepoints            = None,
#                                                                                                n_changepoints          = 25,
#                                                                                                changepoint_range       = 0.8,
#                                                                                                yearly_seasonality      = "auto",
#                                                                                                weekly_seasonality      = "auto",
#                                                                                                daily_seasonality       = "auto",
#                                                                                                holidays                = None,
#                                                                                                seasonality_mode        = "additive",
#                                                                                                seasonality_prior_scale = 10.0,
#                                                                                                holidays_prior_scale    = 10.0,
#                                                                                                changepoint_prior_scale = 0.05,
#                                                                                                mcmc_samples            = 0,
#                                                                                                interval_width          = 0.80,
#                                                                                                uncertainty_samples     = 1000,
#                                                                                                stan_backend            = None),
#                                                            "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
#                                                                                                                compute_full_tree  = 'auto', 
#                                                                                                             #    connectivity       = None, # KHONG SUA
#                                                                                                                distance_threshold = None, 
#                                                                                                             #    linkage            = 'ward', # KHONG SUA
#                                                                                                                memory             = None, 
#                                                                                                                metric             = 'euclidean', 
#                                                                                                                n_clusters         = 3),
#                                                            "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
#                                                                                                                        min_samples                    = None,
#                                                                                                                        metric                         = 'euclidean',
#                                                                                                                        alpha                          = 1.0,
#                                                                                                                        p                              = None,
#                                                                                                                        algorithm                      = 'best',
#                                                                                                                        leaf_size                      = 40,
#                                                                                                                        memory                         = Memory(location=None),
#                                                                                                                        approx_min_span_tree           = True,
#                                                                                                                        gen_min_span_tree              = False,
#                                                                                                                        core_dist_n_jobs               = 1,
#                                                                                                                        cluster_selection_method       = 'eom',
#                                                                                                                        allow_single_cluster           = False,
#                                                                                                                        prediction_data                = False,
#                                                                                                                        match_reference_implementation = False),
#                                                            "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
#                                                            }),
#                                       factor       = 1.5,   # Stick with Prophet model
#                                       window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
#                                       dendrogram   = False, # Stick with Agglomerative Clustering model
#                                       display      = False,
#                                       start_time   = "1990-01-01 00:00:00+07:00",
#                                       end_time     = "2025-01-01 00:00:00+07:00",
#                                       freq         = "1H")

handle

In [ ]:
station["NB"] = dataset.handle_feature_outliers_over_time(data         = station["NB"],
                                                          data_cols    = numerical_features,
                                                          station_name = "NoiBai",
                                                          method       = "machine_learning",
                                                          models       = dict({
                                                                               "IsolationForest"         : IsolationForest(bootstrap     = False, 
                                                                                                                           contamination = "auto", 
                                                                                                                           max_features  = 1.0, 
                                                                                                                           max_samples   = "auto", 
                                                                                                                           n_estimators  = 100, 
                                                                                                                           n_jobs        = None, 
                                                                                                                           random_state  = None, 
                                                                                                                           verbose       = 0, 
                                                                                                                           warm_start    = False),
                                                                               "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
                                                                                                                              contamination = "auto", 
                                                                                                                              leaf_size     = 30, 
                                                                                                                              metric        = "minkowski", 
                                                                                                                              metric_params = None, 
                                                                                                                              n_jobs        = None, 
                                                                                                                              n_neighbors   = 20, 
                                                                                                                              novelty       = False, 
                                                                                                                              p             = 2),
                                                                               "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
                                                                                                                   changepoints            = None,
                                                                                                                   n_changepoints          = 25,
                                                                                                                   changepoint_range       = 0.8,
                                                                                                                   yearly_seasonality      = "auto",
                                                                                                                   weekly_seasonality      = "auto",
                                                                                                                   daily_seasonality       = "auto",
                                                                                                                   holidays                = None,
                                                                                                                   seasonality_mode        = "additive",
                                                                                                                   seasonality_prior_scale = 10.0,
                                                                                                                   holidays_prior_scale    = 10.0,
                                                                                                                   changepoint_prior_scale = 0.05,
                                                                                                                   mcmc_samples            = 0,
                                                                                                                   interval_width          = 0.80,
                                                                                                                   uncertainty_samples     = 1000,
                                                                                                                   stan_backend            = None),
                                                                               "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
                                                                                                                                   compute_full_tree  = 'auto', 
                                                                                                                                #    connectivity       = None, # KHONG SUA
                                                                                                                                   distance_threshold = None, 
                                                                                                                                #    linkage            = 'ward', # KHONG SUA
                                                                                                                                   memory             = None, 
                                                                                                                                   metric             = 'euclidean', 
                                                                                                                                   n_clusters         = 3),
                                                                               "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
                                                                                                                       min_samples                    = None,
                                                                                                                       metric                         = 'euclidean',
                                                                                                                       alpha                          = 1.0,
                                                                                                                       p                              = None,
                                                                                                                       algorithm                      = 'best',
                                                                                                                       leaf_size                      = 40,
                                                                                                                       memory                         = Memory(location=None),
                                                                                                                       approx_min_span_tree           = True,
                                                                                                                       gen_min_span_tree              = False,
                                                                                                                       core_dist_n_jobs               = 1,
                                                                                                                       cluster_selection_method       = 'eom',
                                                                                                                       allow_single_cluster           = False,
                                                                                                                       prediction_data                = False,
                                                                                                                       match_reference_implementation = False),
                                                                               "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
                                                                               }),
                                                          factor       = 1.5,   # Stick with Prophet model
                                                          window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
                                                          dendrogram   = False, # Stick with Agglomerative Clustering model
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">Quy Nhon</span>

detect

In [ ]:
# plots.plot_feature_outliers_over_time(data         = station["QN"],
#                                       data_cols    = numerical_features,
#                                       station_name = "QuyNhon",
#                                       method       = "machine_learning",
#                                       models       = dict({
#                                                            "IsolationForest"         : IsolationForest(bootstrap     = False, 
#                                                                                                        contamination = "auto", 
#                                                                                                        max_features  = 1.0, 
#                                                                                                        max_samples   = "auto", 
#                                                                                                        n_estimators  = 100, 
#                                                                                                        n_jobs        = None, 
#                                                                                                        random_state  = None, 
#                                                                                                        verbose       = 0, 
#                                                                                                        warm_start    = False),
#                                                            "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
#                                                                                                           contamination = "auto", 
#                                                                                                           leaf_size     = 30, 
#                                                                                                           metric        = "minkowski", 
#                                                                                                           metric_params = None, 
#                                                                                                           n_jobs        = None, 
#                                                                                                           n_neighbors   = 20, 
#                                                                                                           novelty       = False, 
#                                                                                                           p             = 2),
#                                                            "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
#                                                                                                changepoints            = None,
#                                                                                                n_changepoints          = 25,
#                                                                                                changepoint_range       = 0.8,
#                                                                                                yearly_seasonality      = "auto",
#                                                                                                weekly_seasonality      = "auto",
#                                                                                                daily_seasonality       = "auto",
#                                                                                                holidays                = None,
#                                                                                                seasonality_mode        = "additive",
#                                                                                                seasonality_prior_scale = 10.0,
#                                                                                                holidays_prior_scale    = 10.0,
#                                                                                                changepoint_prior_scale = 0.05,
#                                                                                                mcmc_samples            = 0,
#                                                                                                interval_width          = 0.80,
#                                                                                                uncertainty_samples     = 1000,
#                                                                                                stan_backend            = None),
#                                                            "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
#                                                                                                                compute_full_tree  = 'auto', 
#                                                                                                             #    connectivity       = None, # KHONG SUA
#                                                                                                                distance_threshold = None, 
#                                                                                                             #    linkage            = 'ward', # KHONG SUA
#                                                                                                                memory             = None, 
#                                                                                                                metric             = 'euclidean', 
#                                                                                                                n_clusters         = 3),
#                                                            "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
#                                                                                                                        min_samples                    = None,
#                                                                                                                        metric                         = 'euclidean',
#                                                                                                                        alpha                          = 1.0,
#                                                                                                                        p                              = None,
#                                                                                                                        algorithm                      = 'best',
#                                                                                                                        leaf_size                      = 40,
#                                                                                                                        memory                         = Memory(location=None),
#                                                                                                                        approx_min_span_tree           = True,
#                                                                                                                        gen_min_span_tree              = False,
#                                                                                                                        core_dist_n_jobs               = 1,
#                                                                                                                        cluster_selection_method       = 'eom',
#                                                                                                                        allow_single_cluster           = False,
#                                                                                                                        prediction_data                = False,
#                                                                                                                        match_reference_implementation = False),
#                                                            "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
#                                                            }),
#                                       factor       = 1.5,   # Stick with Prophet model
#                                       window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
#                                       dendrogram   = False, # Stick with Agglomerative Clustering model
#                                       display      = False,
#                                       start_time   = "1990-01-01 00:00:00+07:00",
#                                       end_time     = "2025-01-01 00:00:00+07:00",
#                                       freq         = "1H")

handle

In [ ]:
station["QN"] = dataset.handle_feature_outliers_over_time(data         = station["QN"],
                                                          data_cols    = numerical_features,
                                                          station_name = "QuyNhon",
                                                          method       = "machine_learning",
                                                          models       = dict({
                                                                               "IsolationForest"         : IsolationForest(bootstrap     = False, 
                                                                                                                           contamination = "auto", 
                                                                                                                           max_features  = 1.0, 
                                                                                                                           max_samples   = "auto", 
                                                                                                                           n_estimators  = 100, 
                                                                                                                           n_jobs        = None, 
                                                                                                                           random_state  = None, 
                                                                                                                           verbose       = 0, 
                                                                                                                           warm_start    = False),
                                                                               "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
                                                                                                                              contamination = "auto", 
                                                                                                                              leaf_size     = 30, 
                                                                                                                              metric        = "minkowski", 
                                                                                                                              metric_params = None, 
                                                                                                                              n_jobs        = None, 
                                                                                                                              n_neighbors   = 20, 
                                                                                                                              novelty       = False, 
                                                                                                                              p             = 2),
                                                                               "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
                                                                                                                   changepoints            = None,
                                                                                                                   n_changepoints          = 25,
                                                                                                                   changepoint_range       = 0.8,
                                                                                                                   yearly_seasonality      = "auto",
                                                                                                                   weekly_seasonality      = "auto",
                                                                                                                   daily_seasonality       = "auto",
                                                                                                                   holidays                = None,
                                                                                                                   seasonality_mode        = "additive",
                                                                                                                   seasonality_prior_scale = 10.0,
                                                                                                                   holidays_prior_scale    = 10.0,
                                                                                                                   changepoint_prior_scale = 0.05,
                                                                                                                   mcmc_samples            = 0,
                                                                                                                   interval_width          = 0.80,
                                                                                                                   uncertainty_samples     = 1000,
                                                                                                                   stan_backend            = None),
                                                                               "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
                                                                                                                                   compute_full_tree  = 'auto', 
                                                                                                                                #    connectivity       = None, # KHONG SUA
                                                                                                                                   distance_threshold = None, 
                                                                                                                                #    linkage            = 'ward', # KHONG SUA
                                                                                                                                   memory             = None, 
                                                                                                                                   metric             = 'euclidean', 
                                                                                                                                   n_clusters         = 3),
                                                                               "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
                                                                                                                       min_samples                    = None,
                                                                                                                       metric                         = 'euclidean',
                                                                                                                       alpha                          = 1.0,
                                                                                                                       p                              = None,
                                                                                                                       algorithm                      = 'best',
                                                                                                                       leaf_size                      = 40,
                                                                                                                       memory                         = Memory(location=None),
                                                                                                                       approx_min_span_tree           = True,
                                                                                                                       gen_min_span_tree              = False,
                                                                                                                       core_dist_n_jobs               = 1,
                                                                                                                       cluster_selection_method       = 'eom',
                                                                                                                       allow_single_cluster           = False,
                                                                                                                       prediction_data                = False,
                                                                                                                       match_reference_implementation = False),
                                                                               "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
                                                                               }),
                                                          factor       = 1.5,   # Stick with Prophet model
                                                          window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
                                                          dendrogram   = False, # Stick with Agglomerative Clustering model
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">Thanh Hoa</span>

detect

In [ ]:
# plots.plot_feature_outliers_over_time(data         = station["TH"],
#                                       data_cols    = numerical_features,
#                                       station_name = "ThanhHoa",
#                                       method       = "machine_learning",
#                                       models       = dict({
#                                                            "IsolationForest"         : IsolationForest(bootstrap     = False, 
#                                                                                                        contamination = "auto", 
#                                                                                                        max_features  = 1.0, 
#                                                                                                        max_samples   = "auto", 
#                                                                                                        n_estimators  = 100, 
#                                                                                                        n_jobs        = None, 
#                                                                                                        random_state  = None, 
#                                                                                                        verbose       = 0, 
#                                                                                                        warm_start    = False),
#                                                            "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
#                                                                                                           contamination = "auto", 
#                                                                                                           leaf_size     = 30, 
#                                                                                                           metric        = "minkowski", 
#                                                                                                           metric_params = None, 
#                                                                                                           n_jobs        = None, 
#                                                                                                           n_neighbors   = 20, 
#                                                                                                           novelty       = False, 
#                                                                                                           p             = 2),
#                                                            "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
#                                                                                                changepoints            = None,
#                                                                                                n_changepoints          = 25,
#                                                                                                changepoint_range       = 0.8,
#                                                                                                yearly_seasonality      = "auto",
#                                                                                                weekly_seasonality      = "auto",
#                                                                                                daily_seasonality       = "auto",
#                                                                                                holidays                = None,
#                                                                                                seasonality_mode        = "additive",
#                                                                                                seasonality_prior_scale = 10.0,
#                                                                                                holidays_prior_scale    = 10.0,
#                                                                                                changepoint_prior_scale = 0.05,
#                                                                                                mcmc_samples            = 0,
#                                                                                                interval_width          = 0.80,
#                                                                                                uncertainty_samples     = 1000,
#                                                                                                stan_backend            = None),
#                                                            "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
#                                                                                                                compute_full_tree  = 'auto', 
#                                                                                                             #    connectivity       = None, # KHONG SUA
#                                                                                                                distance_threshold = None, 
#                                                                                                             #    linkage            = 'ward', # KHONG SUA
#                                                                                                                memory             = None, 
#                                                                                                                metric             = 'euclidean', 
#                                                                                                                n_clusters         = 3),
#                                                            "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
#                                                                                                                        min_samples                    = None,
#                                                                                                                        metric                         = 'euclidean',
#                                                                                                                        alpha                          = 1.0,
#                                                                                                                        p                              = None,
#                                                                                                                        algorithm                      = 'best',
#                                                                                                                        leaf_size                      = 40,
#                                                                                                                        memory                         = Memory(location=None),
#                                                                                                                        approx_min_span_tree           = True,
#                                                                                                                        gen_min_span_tree              = False,
#                                                                                                                        core_dist_n_jobs               = 1,
#                                                                                                                        cluster_selection_method       = 'eom',
#                                                                                                                        allow_single_cluster           = False,
#                                                                                                                        prediction_data                = False,
#                                                                                                                        match_reference_implementation = False),
#                                                            "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
#                                                            }),
#                                       factor       = 1.5,   # Stick with Prophet model
#                                       window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
#                                       dendrogram   = False, # Stick with Agglomerative Clustering model
#                                       display      = False,
#                                       start_time   = "1990-01-01 00:00:00+07:00",
#                                       end_time     = "2025-01-01 00:00:00+07:00",
#                                       freq         = "1H")

handle

In [ ]:
station["TH"] = dataset.handle_feature_outliers_over_time(data         = station["TH"],
                                                          data_cols    = numerical_features,
                                                          station_name = "ThanhHoa",
                                                          method       = "machine_learning",
                                                          models       = dict({
                                                                               "IsolationForest"         : IsolationForest(bootstrap     = False, 
                                                                                                                           contamination = "auto", 
                                                                                                                           max_features  = 1.0, 
                                                                                                                           max_samples   = "auto", 
                                                                                                                           n_estimators  = 100, 
                                                                                                                           n_jobs        = None, 
                                                                                                                           random_state  = None, 
                                                                                                                           verbose       = 0, 
                                                                                                                           warm_start    = False),
                                                                               "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
                                                                                                                              contamination = "auto", 
                                                                                                                              leaf_size     = 30, 
                                                                                                                              metric        = "minkowski", 
                                                                                                                              metric_params = None, 
                                                                                                                              n_jobs        = None, 
                                                                                                                              n_neighbors   = 20, 
                                                                                                                              novelty       = False, 
                                                                                                                              p             = 2),
                                                                               "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
                                                                                                                   changepoints            = None,
                                                                                                                   n_changepoints          = 25,
                                                                                                                   changepoint_range       = 0.8,
                                                                                                                   yearly_seasonality      = "auto",
                                                                                                                   weekly_seasonality      = "auto",
                                                                                                                   daily_seasonality       = "auto",
                                                                                                                   holidays                = None,
                                                                                                                   seasonality_mode        = "additive",
                                                                                                                   seasonality_prior_scale = 10.0,
                                                                                                                   holidays_prior_scale    = 10.0,
                                                                                                                   changepoint_prior_scale = 0.05,
                                                                                                                   mcmc_samples            = 0,
                                                                                                                   interval_width          = 0.80,
                                                                                                                   uncertainty_samples     = 1000,
                                                                                                                   stan_backend            = None),
                                                                               "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
                                                                                                                                   compute_full_tree  = 'auto', 
                                                                                                                                #    connectivity       = None, # KHONG SUA
                                                                                                                                   distance_threshold = None, 
                                                                                                                                #    linkage            = 'ward', # KHONG SUA
                                                                                                                                   memory             = None, 
                                                                                                                                   metric             = 'euclidean', 
                                                                                                                                   n_clusters         = 3),
                                                                               "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
                                                                                                                       min_samples                    = None,
                                                                                                                       metric                         = 'euclidean',
                                                                                                                       alpha                          = 1.0,
                                                                                                                       p                              = None,
                                                                                                                       algorithm                      = 'best',
                                                                                                                       leaf_size                      = 40,
                                                                                                                       memory                         = Memory(location=None),
                                                                                                                       approx_min_span_tree           = True,
                                                                                                                       gen_min_span_tree              = False,
                                                                                                                       core_dist_n_jobs               = 1,
                                                                                                                       cluster_selection_method       = 'eom',
                                                                                                                       allow_single_cluster           = False,
                                                                                                                       prediction_data                = False,
                                                                                                                       match_reference_implementation = False),
                                                                               "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
                                                                               }),
                                                          factor       = 1.5,   # Stick with Prophet model
                                                          window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
                                                          dendrogram   = False, # Stick with Agglomerative Clustering model
                                                          display      = True,
                                                          start_time   = "1990-01-01 00:00:00+07:00",
                                                          end_time     = "2025-01-01 00:00:00+07:00",
                                                          freq         = "1H")

evaluate

##### <span style="color:tomato">TSN</span>

detect

In [ ]:
# plots.plot_feature_outliers_over_time(data         = station["TSN"],
#                                       data_cols    = numerical_features,
#                                       station_name = "TSN",
#                                       method       = "machine_learning",
#                                       models       = dict({
#                                                            "IsolationForest"         : IsolationForest(bootstrap     = False, 
#                                                                                                        contamination = "auto", 
#                                                                                                        max_features  = 1.0, 
#                                                                                                        max_samples   = "auto", 
#                                                                                                        n_estimators  = 100, 
#                                                                                                        n_jobs        = None, 
#                                                                                                        random_state  = None, 
#                                                                                                        verbose       = 0, 
#                                                                                                        warm_start    = False),
#                                                            "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
#                                                                                                           contamination = "auto", 
#                                                                                                           leaf_size     = 30, 
#                                                                                                           metric        = "minkowski", 
#                                                                                                           metric_params = None, 
#                                                                                                           n_jobs        = None, 
#                                                                                                           n_neighbors   = 20, 
#                                                                                                           novelty       = False, 
#                                                                                                           p             = 2),
#                                                            "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
#                                                                                                changepoints            = None,
#                                                                                                n_changepoints          = 25,
#                                                                                                changepoint_range       = 0.8,
#                                                                                                yearly_seasonality      = "auto",
#                                                                                                weekly_seasonality      = "auto",
#                                                                                                daily_seasonality       = "auto",
#                                                                                                holidays                = None,
#                                                                                                seasonality_mode        = "additive",
#                                                                                                seasonality_prior_scale = 10.0,
#                                                                                                holidays_prior_scale    = 10.0,
#                                                                                                changepoint_prior_scale = 0.05,
#                                                                                                mcmc_samples            = 0,
#                                                                                                interval_width          = 0.80,
#                                                                                                uncertainty_samples     = 1000,
#                                                                                                stan_backend            = None),
#                                                            "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
#                                                                                                                compute_full_tree  = 'auto', 
#                                                                                                             #    connectivity       = None, # KHONG SUA
#                                                                                                                distance_threshold = None, 
#                                                                                                             #    linkage            = 'ward', # KHONG SUA
#                                                                                                                memory             = None, 
#                                                                                                                metric             = 'euclidean', 
#                                                                                                                n_clusters         = 3),
#                                                            "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
#                                                                                                                        min_samples                    = None,
#                                                                                                                        metric                         = 'euclidean',
#                                                                                                                        alpha                          = 1.0,
#                                                                                                                        p                              = None,
#                                                                                                                        algorithm                      = 'best',
#                                                                                                                        leaf_size                      = 40,
#                                                                                                                        memory                         = Memory(location=None),
#                                                                                                                        approx_min_span_tree           = True,
#                                                                                                                        gen_min_span_tree              = False,
#                                                                                                                        core_dist_n_jobs               = 1,
#                                                                                                                        cluster_selection_method       = 'eom',
#                                                                                                                        allow_single_cluster           = False,
#                                                                                                                        prediction_data                = False,
#                                                                                                                        match_reference_implementation = False),
#                                                            "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
#                                                            }),
#                                       factor       = 1.5,   # Stick with Prophet model
#                                       window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
#                                       dendrogram   = False, # Stick with Agglomerative Clustering model
#                                       display      = False,
#                                       start_time   = "1990-01-01 00:00:00+07:00",
#                                       end_time     = "2025-01-01 00:00:00+07:00",
#                                       freq         = "1H")

handle

In [ ]:
station["TSN"] = dataset.handle_feature_outliers_over_time(data         = station["TSN"],
                                                           data_cols    = numerical_features,
                                                           station_name = "TSN",
                                                           method       = "machine_learning",
                                                           models       = dict({
                                                                                "IsolationForest"         : IsolationForest(bootstrap     = False, 
                                                                                                                            contamination = "auto", 
                                                                                                                            max_features  = 1.0, 
                                                                                                                            max_samples   = "auto", 
                                                                                                                            n_estimators  = 100, 
                                                                                                                            n_jobs        = None, 
                                                                                                                            random_state  = None, 
                                                                                                                            verbose       = 0, 
                                                                                                                            warm_start    = False),
                                                                                "LocalOutlierFactor"      : LocalOutlierFactor(algorithm     = "auto", 
                                                                                                                               contamination = "auto", 
                                                                                                                               leaf_size     = 30, 
                                                                                                                               metric        = "minkowski", 
                                                                                                                               metric_params = None, 
                                                                                                                               n_jobs        = None, 
                                                                                                                               n_neighbors   = 20, 
                                                                                                                               novelty       = False, 
                                                                                                                               p             = 2),
                                                                                "Prophet"                 : Prophet(growth                  = "linear", # SLOW!!!
                                                                                                                    changepoints            = None,
                                                                                                                    n_changepoints          = 25,
                                                                                                                    changepoint_range       = 0.8,
                                                                                                                    yearly_seasonality      = "auto",
                                                                                                                    weekly_seasonality      = "auto",
                                                                                                                    daily_seasonality       = "auto",
                                                                                                                    holidays                = None,
                                                                                                                    seasonality_mode        = "additive",
                                                                                                                    seasonality_prior_scale = 10.0,
                                                                                                                    holidays_prior_scale    = 10.0,
                                                                                                                    changepoint_prior_scale = 0.05,
                                                                                                                    mcmc_samples            = 0,
                                                                                                                    interval_width          = 0.80,
                                                                                                                    uncertainty_samples     = 1000,
                                                                                                                    stan_backend            = None),
                                                                                "AgglomerativeClustering" : AgglomerativeClustering(compute_distances  = False,  # SLOW!!!
                                                                                                                                    compute_full_tree  = 'auto', 
                                                                                                                                 #    connectivity       = None, # KHONG SUA
                                                                                                                                    distance_threshold = None, 
                                                                                                                                 #    linkage            = 'ward', # KHONG SUA
                                                                                                                                    memory             = None, 
                                                                                                                                    metric             = 'euclidean', 
                                                                                                                                    n_clusters         = 3),
                                                                                "HDBSCAN"                  : HDBSCAN(min_cluster_size               = 5,
                                                                                                                       min_samples                    = None,
                                                                                                                       metric                         = 'euclidean',
                                                                                                                       alpha                          = 1.0,
                                                                                                                       p                              = None,
                                                                                                                       algorithm                      = 'best',
                                                                                                                       leaf_size                      = 40,
                                                                                                                       memory                         = Memory(location=None),
                                                                                                                       approx_min_span_tree           = True,
                                                                                                                       gen_min_span_tree              = False,
                                                                                                                       core_dist_n_jobs               = 1,
                                                                                                                       cluster_selection_method       = 'eom',
                                                                                                                       allow_single_cluster           = False,
                                                                                                                       prediction_data                = False,
                                                                                                                       match_reference_implementation = False),
                                                                                "VanillaAutoencoder"      : "Dell bt ghi gi o day de tuning @@@"
                                                                                }),
                                                           factor       = 1.5,   # Stick with Prophet model
                                                           window_size  = 10,    # Stick with Agglomerative Clustering & DBSCAN model
                                                           dendrogram   = False, # Stick with Agglomerative Clustering model
                                                           display      = True,
                                                           start_time   = "1990-01-01 00:00:00+07:00",
                                                           end_time     = "2025-01-01 00:00:00+07:00",
                                                           freq         = "1H")

evaluate

In [ ]:
# import os
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import tensorflow as tf
# pd.options.mode.chained_assignment = None
# from matplotlib.pylab import rcParams
# import importlib
# importlib.reload(dataset)

# %matplotlib inline

# sns.set_theme(style='whitegrid', palette='muted')
# rcParams['figure.figsize'] = 14, 8
# np.random.seed(1)
# tf.random.set_seed(1)

# df = station["TSN"]
# df.head()

In [ ]:
# for feature in numerical_features:
#     plt.figure(figsize=(14, 6))
#     sns.lineplot(data=df, x='time', y=feature, label=feature)
#     plt.xlabel('Time')
#     plt.ylabel('Value')
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

In [ ]:
# train_size = int(len(df) * 0.8)
# test_size = len(df) - train_size
# train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
# print(train.shape, test.shape)

In [ ]:
# detector_sp = dataset.LSTMAutoencoder()
# detector_tcc = dataset.LSTMAutoencoder()
# detector_tp = dataset.LSTMAutoencoder()
# detector_u10 = dataset.LSTMAutoencoder()
# detector_v10 = dataset.LSTMAutoencoder()

In [ ]:
# detector_sp.prepare_data(train, test, 'sp', 24)
# detector_tcc.prepare_data(train, test, 'tcc', 24)
# detector_tp.prepare_data(train, test, 'tp', 24)
# detector_u10.prepare_data(train, test, 'u10', 24)
# detector_v10.prepare_data(train, test, 'v10', 24)

In [ ]:
# detector_sp.train(epochs=20, batch_size=256, model_path='/TSN/sp')
# detector_tcc.train(epochs=20, batch_size=256, model_path='/TSN/tcc')
# detector_tp.train(epochs=20, batch_size=256, model_path='/TSN/tp')
# detector_u10.train(epochs=20, batch_size=256, model_path='/TSN/u10')
# detector_v10.train(epochs=20, batch_size=256, model_path='/TSN/v10')

In [ ]:
# full_data=df[['time', 'sp']]
# detector_sp.detect_anomalies(full_data)

In [ ]:
# detector_sp.plot_anomaly_scores(full_data)

In [ ]:
# detector_sp.reconstruct_and_replace_anomalies()

In [ ]:
# detector_sp.plot_reconstruct_result()

In [ ]:
# full_data=df[['time', 'tcc']]
# detector_tcc.detect_anomalies(full_data)

In [ ]:
# detector_tcc.plot_anomaly_scores(full_data)

In [ ]:
# detector_tcc.reconstruct_and_replace_anomalies()

In [ ]:
# detector_tcc.plot_reconstruct_result()

In [ ]:
# full_data=df[['time', 'tp']]
# detector_tp.detect_anomalies(full_data)

In [ ]:
# detector_tp.plot_anomaly_scores(full_data)

In [ ]:
# detector_tp.reconstruct_and_replace_anomalies()

In [ ]:
# detector_tp.plot_reconstruct_result()

In [ ]:
# full_data=df[['time', 'u10']]
# detector_u10.detect_anomalies(full_data)

In [ ]:
# detector_u10.plot_anomaly_scores(full_data)

In [ ]:
# detector_u10.reconstruct_and_replace_anomalies()

In [ ]:
# detector_u10.plot_reconstruct_result()

In [ ]:
# full_data=df[['time', 'v10']]
# detector_v10.detect_anomalies(full_data)

In [ ]:
# detector_v10.plot_anomaly_scores(full_data)

In [ ]:
# detector_v10.reconstruct_and_replace_anomalies()

In [ ]:
# detector_v10.plot_reconstruct_result()

#### <span style="color:green">5.2. Feature Engineering</span>

##### <span style="color:tomato">Tính hướng gió, vận tốc gió (gửi file Excel cô tính cho nhóm)</span>

In [ ]:
for name, df in station.items():
    # Convert data u10, v10 sang dang m/s
    u_unit = df["u10"].to_numpy() * units("m/s")
    v_unit = df["v10"].to_numpy() * units("m/s")
    
    # Tinh wind direction, huong gio thoi la from
    wind_direction_deg = metpy.calc.wind_direction(u          = u_unit,
                                                   v          = v_unit,
                                                   convention = "from")
    
    # Tinh wind speed
    wind_speed = metpy.calc.wind_speed(u = u_unit,
                                       v = v_unit)
    
    # Chuyen data wind speed tu dang series sang dang dataframe
    wind_speed         = pd.DataFrame(data = wind_speed,         columns = ["wind_speed"])
    wind_direction_deg = pd.DataFrame(data = wind_direction_deg, columns = ["wind_direction_deg"])
    
    station[name]["wind_speed"]         = wind_speed
    station[name]["wind_direction_deg"] = wind_direction_deg

In [ ]:
station["CaMau"]

##### <span style="color:tomato">Time-based features (hour, day, month, season…)</span>

In [ ]:
for name, df in station.items():
    station[name] = features.extract_time_features(df)

In [ ]:
station["CaMau"]

##### <span style="color:tomato">Arrange columns</span>

In [ ]:
station["CaMau"].columns

In [ ]:
cols_pos = list(['time', 'latitude', 'longitude', 'ymd', 'year', 'month', 'day',
                 'number', 'step', 'surface', 'sp', 'tcc', 'tp', 'u10', 'v10', 'wind_speed', 'wind_direction_deg'])

for name, df in station.items():
    station[name] = df[cols_pos]

In [ ]:
station["CaMau"]

#### <span style="color:green">5.3. Save data</span>

In [ ]:
station["TSN"]

In [ ]:
# Tạo thư mục nếu chưa có
os.makedirs("../data/processed/datasets", exist_ok=True)

# Duyệt từng station và lưu file
for name, df in station.items():
    file_name = f"../data/processed/datasets/{name}90.24_cleaned.csv"
    df.to_csv(file_name, index=False)
    print(f"Đã lưu: {file_name}")